# Workflow: Processing raw data to pandas DataFrame

In [ ]:
import pandas as pd
import numpy as np
import xarray as xr
import os
import matplotlib.pyplot as plt
import scipy
import scipy.stats as stats
import seaborn as sns
xr.set_options(display_style='html')

# To install netCDF4 for .nc files (does not come pre-loaded in CO)
!pip install netCDF4
from netCDF4 import Dataset
import netCDF4

import IPython
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual


### To mount google drive in runtime to access files in the drive so you can access files
from google.colab import drive
drive.mount('/content/drive')

home_dir = '/content/drive/My Drive/100yr-Ensemble-Data/Spatial-and-Annual-Averages/'
gnu_dir = home_dir+'GNU-runs/'
intel_dir = home_dir+'Intel-runs/'
test_dir = home_dir+'Test-runs/'

### Use local directory to save a copy to your folder
local_dir = '/content/drive/My Drive/Ensemble_project/'

# Set tot True if we want to drop Intel Simulation 115
drop_115 = True
drop_000 = True
get_data = False
remove_var = True

In [ ]:
# Credits to Curtis
def approx_z_cleaning(input_df,my_var,threshold):
  #std_temp = np.std(input_variable)
  #mean_temp = np.mean(input_variable)
  low_percentile = input_df[my_var].quantile(0.025)
  high_percentile = input_df[my_var].quantile(0.975)
  variable_mid = input_df[(input_df[my_var] > low_percentile) & (input_df[my_var] < high_percentile)][my_var]
  mean_mid = variable_mid.mean()
  std_mid = variable_mid.std()
  if std_mid != 0:
    reduced_var = (input_df[my_var] - mean_mid)/std_mid
    bad_data_flags = reduced_var[np.abs(reduced_var) > threshold].index
  return bad_data_flags


In [ ]:
var_df = pd.read_csv(local_dir+'CESM_variable_type.csv')
var_df['my_type'] = var_df['type']
var_df

In total 114 Intel simulations, throw Intel.115 as it is compromised. Throw Intel.006 due to wrong SNOWHICE. 

In [ ]:
### Start empty DataFrame here
if get_data:
  df = pd.DataFrame()

  ### Append all data into this DataFrame
  for my_dir, my_label in zip([gnu_dir,intel_dir,test_dir],['GNU','Intel','Test']):
    for filename in os.listdir(my_dir):
      if filename.endswith("nc"): 
          print(filename)
          simulation = filename.split('.')[-2]
          print(simulation)
          ds = xr.open_dataset(my_dir+filename)
          newdf = ds.to_dataframe()
          newdf = newdf.reset_index(level=[0,1])
          newdf['varNames'] = newdf['varNames'].str.decode("utf-8")


          newdf = newdf.pivot_table(index = ['nyear'],columns=['varNames'],values=['spatialAvgs'])
          newdf.columns = newdf.columns.droplevel(0)
          # Label the simulations with 'GNU','Intel', or 'Test'
          newdf['Label'] = my_label
          newdf['Simulation'] = str(simulation)
          newdf = newdf.reset_index()
          df = df.append(newdf,ignore_index = True)
  df.Simulation = df.Simulation.astype(str)
  if drop_115:
    df.drop(index = df[(df.Label == 'Intel') & (df.Simulation.str.contains('115'))].index,inplace = True)
  if drop_000:
    df.drop(index = df[(df.Label == 'Intel') & (df.Simulation.str.contains('000'))].index,inplace = True)
  
  df.to_csv(local_dir+'ensemble_all.csv')

else:
  df = pd.read_csv(local_dir+'ensemble_all.csv')
  df = df.drop(axis=1,columns=['Unnamed: 0'])
  #df[df.TS < 280] = np.nan
  df['Simulation'] = [str(my_str).zfill(3) for my_str in df['Simulation']]
  if drop_115:
    df.drop(index = df[(df.Label == 'Intel') & (df.Simulation.str.contains('115'))].index,inplace = True)
  if drop_000:
    df.drop(index = df[(df.Label == 'Intel') & (df.Simulation.str.contains('000'))].index,inplace = True)
  

Evolution of distribution across simulation

Credit to (partially): https://stackoverflow.com/questions/58786611/plotly-two-subplots-controlled-by-one-slider

In [ ]:
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# Add histogram data
my_var = 'BURDEN2'
intel_df = df[(df.Label == 'Intel')]
my_test_input = 'rh-min-low'
nsteps = 10
fig = go.Figure()

gnu_df = df[df.Label == 'GNU']



def make_interact_dist(my_var,nsteps,my_test = my_test_input):
  init_year = 0
  test_df = df[df.Simulation == my_test]
  mean_init_intel = intel_df[intel_df.nyear == init_year].mean()[my_var]
  std_init_intel = intel_df[intel_df.nyear == init_year].std()[my_var]
  mean_init_gnu = gnu_df[gnu_df.nyear == init_year].mean()[my_var]
  std_init_gnu = gnu_df[gnu_df.nyear == init_year].std()[my_var]
  fig = make_subplots(rows=1, cols=1,shared_xaxes=True)#, subplot_titles = ['Mean %f' % (mean_init_intel) +', Std %f' % (std_init_intel),'Mean %f' % (mean_init_gnu) +', Std %f' % (std_init_gnu)]


  #xmin = np.nanmin((intel_df.min()[my_var],gnu_df.min()[my_var]))
  #xmax = np.nanmax((intel_df.max()[my_var],gnu_df.max()[my_var]))

  xmin = np.nanmin((intel_df.min()[my_var],gnu_df.min()[my_var],test_df.min()[my_var]))
  xmax = np.nanmax((intel_df.max()[my_var],gnu_df.max()[my_var],test_df.max()[my_var]))
  nbins = 40
  bin_size = (xmax - xmin)/nbins
  xbins = np.linspace(xmin,xmax,nbins+1)

  print(xmin,xmax,np.percentile((xmin,xmax),80))
  intel_ymax = []
  gnu_ymax = []
  intel_mean = []
  intel_std = []
  gnu_mean = []
  gnu_std = []
  for step in np.arange(0, nsteps, 1):
    my_year = step
    
    x1 = np.array(intel_df[intel_df.nyear == my_year][my_var])
    x2 = np.array(gnu_df[gnu_df.nyear == my_year][my_var])
    #x3 = np.array(test_df[test_df.nyear == my_year][my_var])    


    # Create distplot with custom bin_size
    #fig = ff.create_distplot(hist_data, group_labels)#, bin_size=[.05, 0.05]
    counts_intel, bins = np.histogram(x1,bins = xbins,density =True)
    counts_gnu, bins = np.histogram(x2,bins = xbins,density =True)
 
    counts_intel = counts_intel/np.sum(counts_intel)
    counts_gnu = counts_gnu/np.sum(counts_gnu)
    intel_ymax.append(counts_intel)
    gnu_ymax.append(counts_intel)
    intel_mean.append(x1.mean())
    intel_std.append(x1.std())
    gnu_mean.append(x2.mean())
    gnu_std.append(x2.std())
  ### Decide ymax
  ymax_intel = np.nanmax(intel_ymax)
  ymax_gnu = np.nanmax(gnu_ymax)   
  ymax = np.nanmax([ymax_intel,ymax_gnu]) 
  print(intel_std)

  for step in np.arange(0, nsteps, 1):
    my_year = step
    
    x1 = np.array(intel_df[intel_df.nyear == my_year][my_var])
    x2 = np.array(gnu_df[gnu_df.nyear == my_year][my_var])
    x3 = np.array(test_df[(df.Simulation == my_test) & (df.nyear == my_year)][my_var])[0]
    fig.add_trace(go.Histogram(visible=False,x=x1, histnorm='probability',name = 'Intel',xbins=dict(
                      start=xmin,
                      end=xmax,
                      size=bin_size), 
                      autobinx=False
                     ),row=1, col=1)
    
    fig.add_trace(go.Histogram(visible=False,x=x2, histnorm='probability',name = 'GNU',xbins=dict(
                      start=xmin,
                      end=xmax,
                      size=bin_size), 
                      autobinx=False
                     ),row=1, col=1)
    #fig.add_annotation(
    #          visible = False,
    #          x=np.percentile((xmin,xmax),20),
    #          y=ymax_intel*0.95,
    #          text="Mean: %f" % ((x1.mean())),
    #          showarrow= False
    #  )
    #fig.add_annotation(
    #          visible = False,
    #          x=np.percentile((xmin,xmax),20),
    #          y=ymax_intel*0.8,
    #          text="Std: %f" % ((x1.std())),
    #          showarrow= False
    #  )

    fig.add_trace(go.Scatter(visible=False,x=[x3, x3], y=[0,ymax], 
      mode="lines",name = my_test, line=dict(color="black",dash='dash') ),row=1,col=1)
    fig.add_trace(go.Scatter(visible=False,x=[x1.mean(), x1.mean()], y=[0,ymax], 
      mode="lines",name = 'Mean Intel',line=dict(color="blue",dash='dash') ),row=1,col= 1)
    fig.add_trace(go.Scatter(visible=False,x=[x2.mean(), x2.mean()], y=[0,ymax], 
      mode="lines",name = 'Mean GNU',line=dict(color="red",dash='dash') ),row=1,col= 1)
        
    fig.update_traces(opacity=0.75)
    

    #fig['layout']['annotations'][0].update(text='Mean %f' % (x1.mean()) +', Std %f' % (x1.std()))
    #fig['layout']['annotations'][1].update(text='Mean %f' % (x2.mean()) +', Std %f' % (x2.std()))
      




  fig.data[0].visible = True
  fig.data[1].visible = True
  fig.data[2].visible = True
  fig.data[3].visible = True
  fig.data[4].visible = True

  #fig.layout.annotations[0].visible = False
  #fig.layout.annotations[1].visible = False

  # Create and add slider
  years = []
  
  for i in range(0, len(fig.data), 5):
      year = dict(
          method="update",
          label = 'Year {}'.format(int(i/5)+1),
          args=[{"visible": [False] * len(fig.data)},
                {"title": "Distribution across ensemble for year " + str(int(i/5)+1)},
                {"annotation": ['Mean %f' % my_mean for my_mean in (intel_mean)]}
                ],  # layout attribute
      )
      year["args"][0]["visible"][i] =True  # Toggle i'th trace to "visible"
      year["args"][0]["visible"][i+1] =True  # Toggle i'th trace to "visible"
      year["args"][0]["visible"][i+2] =True  # Toggle i'th trace to "visible"
      year["args"][0]["visible"][i+3] =True  # Toggle i'th trace to "visible"
      year["args"][0]["visible"][i+4] =True  # Toggle i'th trace to "visible"
      #for new_idx in range(len(fig.layout.annotations)):
      #  fig.layout.annotations[new_idx].visible = False

      years.append(year)

  sliders = [dict(
      active=0,
      currentvalue={"prefix": "This is "},
      pad={"t": nsteps},
      steps=years
  )]

  fig.update_layout(
      sliders=sliders
  )

  fig.update_xaxes(range=[xmin, xmax])
  fig.update_yaxes(range=[0, ymax*1.1],row=1,col=1)
  #fig.update_yaxes(range=[0, ymax_gnu*1.2],row=2,col=1)


  fig.show()

  return fig
my_fig = make_interact_dist(my_var = 'SNOWHLND',nsteps = 10,my_test = 'rh-min-low')
my_fig.write_html(local_dir+"/interactive_distribution.html")

Throw away outliers with 95% in middle: Z values should be smaller than 7-sigma. 

A good test with BURDEN2, first year should be thrown away.

In [ ]:
#var = var_list[90]
remove_outlier = True
df_clean = df.copy()

my_ens = 'Intel'
my_var = 'BURDEN2'
simu_list = np.unique(df_clean[df_clean.Label == my_ens]['Simulation'])
for simu in simu_list:
  input_df = df_clean[(df_clean.Label == my_ens) & (df_clean.Simulation == simu)]
  test_loc = approx_z_cleaning(input_df,my_var,10)
  if remove_outlier:
    #print(test_loc)
    df_clean[(df_clean.Label == my_ens) & (df_clean.Simulation == simu)].loc[test_loc][my_var] = np.nan

In [ ]:
newdf = df[(df.Label == 'GNU') & (df.Simulation == '001')]
newdf.iloc[0].BURDEN2/newdf[1:100].BURDEN2.mean()

In [ ]:
df_clean[df_clean['BURDEN2'].isnull()]#['BURDEN2']


In [ ]:
def enable_plotly_in_cell():
    display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
    init_notebook_mode(connected=False)

In [ ]:
df.columns

In [ ]:
### STATS figure for Liz, 2020/09
fs = 14
sns.set_context("paper", rc={"font.size":14,"axes.titlesize":14,"axes.labelsize":14})   

fig = plt.figure(1,figsize=(10,8))
newdf = df[df.Label.isin(['Intel','GNU'])]
b = sns.lineplot(x = 'nyear',y = 'TS',data = newdf, hue = 'Label', ci = 95)

b.tick_params(labelsize=fs-2)
#b.set_xlabel(fontsize=fs-2)


fig = plt.figure(2,figsize = (8,6))
my_var = 'TS'
my_year = 0
df_x = df[(df.Label == 'Intel') & (df.nyear == my_year)]
df_y = df[(df.Label == 'GNU') & (df.nyear == my_year)]
res = stats.ks_2samp(np.array(df_y[my_var]),np.array(df_x[my_var]))

bins = np.linspace(np.nanmin([df_x[my_var].min(),df_y[my_var].min()]),np.nanmax([df_x[my_var].max(),df_y[my_var].max()]),20)
plt.hist(df_x[my_var], bins=bins, density=True, histtype='step', cumulative=True,
        label='Intel',lw=2)
plt.hist(df_y[my_var], bins=bins, density=True, histtype='step', cumulative=True,
        label='GNU',lw=2)
plt.title('nyear: {my_year}, ks-pvalue: {pval}'.format(my_year= my_year,pval = round(res[1],2)))
plt.legend(loc = 2,frameon=False,fontsize = fs-2)
plt.xlabel('TS (K)',fontsize = fs-2)
plt.ylabel('CDF',fontsize = fs-2)
plt.xticks(fontsize = fs-2)
plt.yticks(fontsize = fs-2)
plt.locator_params(nbins=5)
print('KS-statistics: ',res)


fig = plt.figure(3,figsize = (8,6))
my_var = 'TS'
my_year = 63
df_x = df[(df.Label == 'Intel') & (df.nyear == my_year)]
df_y = df[(df.Label == 'GNU') & (df.nyear == my_year)]
res = stats.ks_2samp(np.array(df_y[my_var]),np.array(df_x[my_var]))

bins = np.linspace(np.nanmin([df_x[my_var].min(),df_y[my_var].min()]),np.nanmax([df_x[my_var].max(),df_y[my_var].max()]),20)
plt.hist(df_x[my_var], bins=bins, density=True, histtype='step', cumulative=True,
        label='Intel',lw=2)
plt.hist(df_y[my_var], bins=bins, density=True, histtype='step', cumulative=True,
        label='GNU',lw=2)
plt.title('nyear: {my_year}, ks-pvalue: {pval}'.format(my_year= my_year,pval = round(res[1],2)))
plt.legend(loc = 2,frameon=False, fontsize = fs-2)
plt.xlabel('TS (K)',fontsize = fs-2)
plt.ylabel('CDF',fontsize = fs-2)
plt.xticks(fontsize = fs-2)
plt.yticks(fontsize = fs-2)
plt.locator_params(nbins=5)


# Workflow: Visualization with pandas

Pick several random Intel runs: 

In [ ]:
### Take 5 random simulation numbers to be shown
rand = np.random.randint(0,115,5)
rand_list = [str(my_rand).zfill(3) for my_rand in rand]
rand_list

In [ ]:
my_df = df.copy()
#print(my_df)
my_df['Simulation'] = my_df['Simulation'].astype(str)
print("1\n",my_df)
my_df = my_df[(my_df.Label == 'Intel') & (my_df['Simulation'].isin(rand_list))]
print("2\n",my_df)
my_df = my_df.melt(id_vars=['nyear','Simulation'],value_vars=['TS'])
print("3\n",my_df)

In [ ]:
miss_df = df[df.isnull().any(axis=1)]
for col,my_bool in zip(df.columns,(df[df.isnull().any(axis=1)].isnull().any(axis=0))):
  if my_bool:
    print('A missing value appear for: ',col)

In [ ]:
miss_df[['nyear','Label','Simulation','TS','AODVIS','CCN3','ICWMR','SNOWHICE','VD01','VV','ncl_a1_SRF']]

In [ ]:
#import seaborn as sns
#sns.lineplot(x="nyear", y="value", data=my_df)

### An animation of heatmap.

In [ ]:
newdf = df[df.Label == 'GNU']
run_list = np.unique(newdf['Simulation'].astype(float))
nrun = len(run_list)
col_df = newdf.drop(axis=1,columns=['nyear','Label','Simulation'])
# Remove constant columns
#col_df = col_df.loc[:,col_df.apply(pd.Series.nunique) != 1]
column_list = col_df.columns
ncol = len(column_list)



Which variables do not change across the ensemble? 

In [ ]:
removed_column_list = list(column_list)
# Remove three variables with bump start: 'OCNFRAC','ICEFRAC','SNOWHICE'
# Remove three constant variables: 'LANDFRAC','PHIS','SOLIN'
#remove_var_list =  ['EMISCLD','OCNFRAC','ICEFRAC','LANDFRAC','PHIS','SOLIN']
remove_var_list =  ['H2O2_SRF','DTWR_H2O2','OCNFRAC','ICEFRAC','LANDFRAC','PHIS','SOLIN','EMISCLD'] #SNOWHLND

for my_col in remove_var_list:
  removed_column_list.remove(my_col)
np.array(removed_column_list,dtype=str)
col_list = column_list.copy()

#if remove_var:
#  column_list = removed_column_list

In [ ]:
import scipy.stats as stats
gb = df[(df.Label == 'Intel')].groupby('nyear')
z_list = []
for my_var in removed_column_list:
  #my_var = removed_column_list[0]
  x = df[(df.Label == 'Intel') & (df.nyear == 0)][my_var].std()
  dist = gb[my_var].std()
  if dist.std() > 0:
    zscore = (x - dist.mean())/dist.std()
    print(zscore)
    if abs(zscore) > 3:
      print(my_var)
    z_list.append(zscore)

In [ ]:
plt.hist(z_list)
z_list = np.array(z_list)
len(z_list[z_list < 0])*1./len(z_list)

#### Each variable


In [ ]:
'''
from plotly.offline import init_notebook_mode, iplot, plot
import plotly.graph_objs as go  

enable_plotly_in_cell()

var_df = newdf.pivot_table(values = ['T'],index = ['nyear'],columns = ['Simulation'])
# remove outliers
fig = go.Figure(data=go.Heatmap(
                    z=var_df))


fig.update_layout(
    showlegend = True,
    width = 700, height = 700,
    autosize = False,xaxis_title="Simulations",yaxis_title="Years" )
fig.show()
'''

#### Z score (mean and standard deviation)


We can also assume a Gaussian distribution and fit each dataset to it. Then we can plot mean and standard deviation. 

Each year, use 114 Intel simulation as whole distribution, plot test runs Z-value. 


In [ ]:
'''
from scipy.stats import norm
import scipy.stats as stats

my_ensemble = 'Intel'
my_var = 'TS'
year_list = np.arange(0,100,1)
nyear = len(year_list)

col_df = newdf
ncol = len(col_df.columns)
all_z_matrix = np.zeros((nyear,ncol,nrun))
simu_list = np.unique(df[df.Label == my_ensemble]['Simulation'])

mean_list = np.zeros(nyear)
std_list = np.zeros(nyear)
#mean_matrix = np.zeros((ncol,nyear))
#std_matrix = np.zeros((ncol,nyear))
for iz,my_simu in enumerate(simu_list):
  z_df = df[(df.Label == my_ensemble) & (df.Simulation == my_simu)]
  #z_df = df[(df.Label == my_ensemble)]
  for iy, col in enumerate(col_df.columns):
    x = z_df[[col]]
    #y = newdf[column_list[iy]]
    mask = np.isfinite(x) #& np.isfinite(y)
    mu, std = norm.fit(x[mask])
    mean_list[ix] = mu
    #mean_matrix[ix,iy] = mu*1./std
    std_list[ix] = std
    #std_matrix[ix,iy] = std

    for ix, year in enumerate(year_list):
      all_z_matrix[ix,iy,iz] = (df[(df.Label == my_ensemble) & (df.nyear == year) & (df.Simulation == my_simu)][my_var] - mu)/std*1.

# Now test with another dataset
#test_df = df[df.Label == 'GNU']
intel_df =df[df.Label == 'Intel']
gnu_dg = df[df.Label == 'GNU'] 
test_df = df[df.Label == 'Test']
test_list=np.unique(test_df.Simulation)
ntest = len(test_list)

'''

In [ ]:
df['ANRAIN'].describe()

In [ ]:
### Plot the all z-values
'''
print(np.shape(all_z_matrix))
enable_plotly_in_cell()

print('This is heatmap of 8 test runs, ',my_var)
print('Testing against ',my_ensemble)
fig = go.Figure(data=go.Heatmap(
                    z=all_z_matrix[:,0,:],x = test_list))

fig.update_layout(
    showlegend = True,
    width = 700, height = 700,
    autosize = False,xaxis_title="Test runs ",yaxis_title="Years" )
fig.show()
'''

Now for each ensemble (GNU,Intel) show z value for each test run

In [ ]:
from scipy.stats import norm
import scipy.stats as stats

my_ensemble = 'GNU'
my_var = 'TS'
year_list = np.arange(0,100,1)
nyear = len(year_list)

col_df = newdf[[my_var]]
ncol = len(col_df.columns)
all_z_matrix = np.zeros((nyear,ncol,nrun))
simu_list = np.unique(df[df.Label == my_ensemble]['Simulation'])

mean_list = np.zeros(nyear)
std_list = np.zeros(nyear)
#mean_matrix = np.zeros((ncol,nyear))
#std_matrix = np.zeros((ncol,nyear))
for ix, year in enumerate(year_list):
  z_df = df[(df.Label == my_ensemble) & (df.nyear == year)]
  #z_df = df[(df.Label == my_ensemble)]
  for iy, col in enumerate(col_df.columns):
    x = z_df[col]
    #y = newdf[column_list[iy]]
    mask = np.isfinite(x) #& np.isfinite(y)
    mu, std = norm.fit(x[mask])
    mean_list[ix] = mu
    #mean_matrix[ix,iy] = mu*1./std
    std_list[ix] = std
    #std_matrix[ix,iy] = std


# Now test with another dataset
#test_df = df[df.Label == 'GNU']
intel_df =df[df.Label == 'Intel']
gnu_dg = df[df.Label == 'GNU'] 
test_df = df[df.Label == 'Test']
test_list=np.unique(test_df.Simulation)
ntest = len(test_list)

#print(test_df)
z_matrix = np.zeros((nyear,ntest))
p_matrix = np.zeros((nyear,ntest))
for itest in range(ntest):
  for iyear in range(nyear):
    newdf = test_df[(test_df.nyear == year_list[iyear]) & (test_df.Simulation == test_list[itest])]
    if len(newdf) == 1:
      z_matrix[iyear,itest] = (newdf[my_var] - mean_list[iyear]) / std_list[iyear]
      p_matrix[iyear,itest] = stats.norm.sf(abs(z_matrix[iyear,itest]))
#p_matrix


In [ ]:
'''
### Plot the one-sided p-value for each test run, my_var
enable_plotly_in_cell()

print('This is heatmap of 8 test runs, ',my_var)
print('Testing against ',my_ensemble)
fig = go.Figure(data=go.Heatmap(
                    z=p_matrix,x = test_list))

fig.update_layout(
    showlegend = True,
    width = 700, height = 700,
    autosize = False,xaxis_title="Test runs ",yaxis_title="Years" )
fig.show()
'''

In [ ]:
fig = plt.figure(1)
plt.hist(p_matrix[:,2], density=True, histtype='stepfilled', alpha=0.2)
res = stats.kstest(p_matrix[:,2],'uniform')
xlim = plt.gca().get_xlim()
ylim = plt.gca().get_ylim()
plt.text(xlim[1]*0.8,ylim[1]*0.8,'p-value: '+str(round(res[1],2)))
res

In [ ]:
### 9 subplots
normalize_y = False
nrow = 3
ncol = 3
fig = plt.figure(figsize=(12,12))
plt.subplots_adjust(wspace = 0.4,hspace= 0.3)

#fig = plt.figure(figsize=(8,8))
var_list = ['TS','TSMN','TSMX','VT','VU','U10','LWCF','BURDEN2','CCN3']

plt.tight_layout()
my_perc = np.arange(1,100,1)
# Set to True to use only 25 simulations 
matched = True 

if not matched:
  gnu_df = df[(df.Label == 'GNU')]
  intel_df = df[(df.Label == 'Intel') ]
  intel_perc = np.nanpercentile(df[(df.Label == 'Intel')][my_var],my_perc)
  all_test_df= df[(df.Label == 'Test') ]
else:
  match_list = [str(simu).zfill(3) for simu in range(25)]
  match_list.remove('000')
  gnu_df = df[(df.Label == 'GNU') & (df.Simulation.isin(match_list))]
  intel_df = df[(df.Label == 'Intel') & (df.Simulation.isin(match_list))]
  all_test_df= df[(df.Label == 'Test') ]
  intel_df = intel_df.dropna(how = 'any')
  gnu_df = gnu_df.dropna(how = 'any')

for irow in range(nrow):
  for icol in range(ncol):
    idx = ncol*irow+icol
    ax = plt.subplot(nrow,ncol,idx+1)
    my_var = var_list[idx]
    
    gb1 = intel_df.groupby('nyear')
    gb2 = gnu_df.groupby('nyear')

    plt.plot(np.arange(0,100),gb1.mean()[my_var],label = 'Intel')
    plt.plot(np.arange(0,100),gb2.mean()[my_var],label = 'GNU')
    plt.xlabel('Year')
    plt.ylabel(my_var)
    if irow == 0 and icol == 2:
      plt.legend(frameon=False)


In [ ]:
#gb.mean()['BURDEN2']

#### How to interpret KS statistics


KS Test is a very powerful way to automatically differentiate samples from a different distribution. kstest function may also be used to check whether the data given follows Normal Distribution or not. It compares the observed versus the expected cumulative relative frequencies of the Normal Distribution. The Kolmogorov-Smirnov test uses the maximal absolute difference between the observed and expected cumulative distribution.

The Null hypothesis used here assumes that the numbers follow the normal distribution.
The functioning of the function remains exactly same. Again it returns statistics and p-value. If the p-value is < alpha, we reject the Null hypothesis.

https://www.geeksforgeeks.org/ml-kolmogorov-smirnov-test/


https://stackoverflow.com/questions/39132469/how-to-interpret-scipy-stats-kstest-and-ks-2samp-to-evaluate-fit-of-data-t

In [ ]:
res.pvalue

How to interpret the results above? 
- Last columns seems much darker (blue/purple), which means p-value is small: More likely to be in from a different distribution. 

- Intended to see whether there's change in time axis: First year more similar to each other? 

### Sanity Check (Quality control of data)

#### QQ-plot


100yr average. Vs gaussian. Vs GNU, VS test. 

Aggregated over 100 years: Intel vs GNU. 

In [ ]:
def qqplot(df, my_var):
    #function from above, but without the color generator
    newdf = df.dropna(subset=[my_var]).copy()
    fig = plt.figure(2,figsize = (8,8))
    my_perc = np.arange(0,100,1)
    # Set to True to use only 25 simulations 
    matched = True 
    color_list = ['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00','#ffff33','#a65628','#f781bf']#['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00']

    if not matched:
      intel_perc = np.nanpercentile(newdf[(newdf.Label == 'Intel')][my_var],my_perc)
    else:
      match_list = [str(simu).zfill(3) for simu in range(25)]
      #match_list = [(simu) for simu in range(25)]
      intel_perc = np.nanpercentile(newdf[(newdf.Label == 'Intel') & (newdf.Simulation.isin(match_list))][my_var],my_perc)
    gnu_perc = np.nanpercentile(newdf[newdf.Label == 'GNU'][my_var],my_perc)

    ax = plt.subplot()
    vmin = np.round(np.nanmin([intel_perc,gnu_perc]),decimals=2)
    vmax = np.round(np.nanmax([intel_perc,gnu_perc]),decimals=2)

    plt.plot([vmin,vmax],[vmin,vmax],'--k')
    plt.plot(intel_perc,gnu_perc,'--b',label = 'GNU')
    plt.xlabel('Intel',fontsize = 12)
    plt.ylabel('GNU/Test',fontsize = 12)

    test_list = np.unique(df[df.Label == 'Test']['Simulation'])
    for my_test,my_color in zip(test_list,color_list):
      test_perc = np.nanpercentile(newdf[(newdf.Simulation == my_test)][my_var],my_perc)
      plt.plot(intel_perc,test_perc,c = my_color,label = my_test,lw = 2)

    ax.set_xticks(np.round(np.linspace(vmin, vmax, 5), 2))
    ax.set_yticks(np.round(np.linspace(vmin, vmax, 5), 2))


    plt.title(my_var)
    plt.legend()
    fig.patch.set_facecolor('xkcd:white')

    return fig
#qqplot(df,'TS')

Test QQ plot against Gaussian CDF. 


In [ ]:
newdf = df.dropna(subset=[my_var]).copy()
fig = plt.figure(2,figsize = (8,8))
my_perc = np.arange(0,100,1)
color_list = ['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00','#ffff33','#a65628','#f781bf']#['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00']

# Set to True to use only 25 simulations 
matched = True 
if not matched:
  intel_perc = np.nanpercentile(newdf[(newdf.Label == 'Intel')][my_var],my_perc)
else:
  match_list = [str(simu).zfill(3) for simu in range(25)]
  #match_list = [(simu) for simu in range(25)]
  intel_perc = np.nanpercentile(newdf[(newdf.Label == 'Intel') & (newdf.Simulation.isin(match_list))][my_var],my_perc)
gnu_perc = np.nanpercentile(newdf[newdf.Label == 'GNU'][my_var],my_perc)

ax = plt.subplot()
vmin = np.round(np.nanmin([intel_perc,gnu_perc]),decimals=2)
vmax = np.round(np.nanmax([intel_perc,gnu_perc]),decimals=2)

plt.plot([vmin,vmax],[vmin,vmax],'--k')
plt.plot(intel_perc,gnu_perc,'--b',label = 'GNU')
plt.xlabel('Intel',fontsize = 12)
plt.ylabel('GNU/Test',fontsize = 12)

test_list = np.unique(df[df.Label == 'Test']['Simulation'])
for my_test,my_color in zip(test_list,color_list):
  test_perc = np.nanpercentile(newdf[(newdf.Simulation == my_test)][my_var],my_perc)
  plt.plot(intel_perc,test_perc,c = my_color,label = my_test,lw = 2)

ax.set_xticks(np.round(np.linspace(vmin, vmax, 5), 2))
ax.set_yticks(np.round(np.linspace(vmin, vmax, 5), 2))


plt.title(my_var)
plt.legend()
fig.patch.set_facecolor('xkcd:white')

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

var_list = column_list

@interact(my_var=var_list)
def make_plot_for(my_var='TS'):
    plot = qqplot(df, my_var)
    #display(plot)


Linear fit of all variables, Intel ensemble last 90 years. 

One problem is that I forced low and high end to match. 

In [ ]:

from scipy.stats import norm
normalize_y = False
def gaussian_qqplot(my_var,normalize_y):
  fig = plt.figure(2,figsize = (8,8))
  my_perc = np.arange(1,100,1)
  # Set to True to use only 25 simulations 
  matched = False 

  if not matched:
    gnu_df = df[(df.Label == 'GNU')]
    intel_df = df[(df.Label == 'Intel') ]
    intel_perc = np.nanpercentile(df[(df.Label == 'Intel')][my_var],my_perc)
  else:
    match_list = [str(simu).zfill(3) for simu in range(25)]
    #match_list.remove('006')
    gnu_df = df[(df.Label == 'GNU') & (df.Simulation.isin(match_list))]
    intel_df = df[(df.Label == 'Intel') & (df.Simulation.isin(match_list))]
  gnu_mean = gnu_df[my_var].mean()
  gnu_std = gnu_df[my_var].std()
  gnu_perc = np.nanpercentile(df[df.Label == 'GNU'][my_var],my_perc)
  if normalize_y:
    gnu_perc = [(my_gnu_perc-gnu_mean)/gnu_std for my_gnu_perc in gnu_perc]
  intel_mean = intel_df[my_var].mean()
  intel_std = intel_df[my_var].std()
  intel_perc = np.nanpercentile(df[(df.Label == 'Intel')][my_var],my_perc)
  if normalize_y:
    intel_perc = [(my_intel_perc-intel_mean)/intel_std for my_intel_perc in intel_perc]
  test_perc = np.nanpercentile(df[(df.Label == 'Test')][my_var],my_perc)

  #gnu_hist = np.histogram(np.array(gnu_df[my_var]),bins = np.linspace(gnu_df[my_var].min(),gnu_df[my_var].max(),101))[0]
  #gnu_hist = gnu_hist/gnu_hist.sum(axis=0, keepdims=True) 
  #gnu_perc = np.cumsum(gnu_hist)
  #intel_hist = np.histogram(np.array(intel_df[my_var]),bins = np.linspace(intel_df[my_var].min(),intel_df[my_var].max(),101))[0]
  #intel_hist = intel_hist/intel_hist.sum(axis=0, keepdims=True) 
  #intel_perc = np.cumsum(intel_hist)

  x = [norm.ppf(my_quant) for my_quant in np.linspace(0.01,0.99,99)]
  #gaussian_perc = norm.cdf(x)
  gaussian_perc = x.copy()

  vmin = np.nanmin([gnu_perc,intel_perc])
  vmax = np.nanmax([gnu_perc,intel_perc])
  plt.plot([np.nanmin(x),np.nanmax(x)],[vmin,vmax],'k--',alpha = 0.5)#,label = 'Gaussian'

  plt.plot(gaussian_perc,gnu_perc,'k',linestyle = 'dashdot',label = 'GNU',zorder = 10)
  plt.plot(gaussian_perc,intel_perc,'k-',label = 'Intel',zorder = 10)


  plt.xlabel('Standard Normal Quantiles',fontsize = 12)
  plt.ylabel('Intel/GNU/Test Quantiles',fontsize = 12)

  test_list = np.unique(df[df.Label == 'Test']['Simulation'])
  for my_test,my_color in zip(test_list,color_list):
    test_df = df[df.Simulation == my_test]
    #test_hist = np.histogram(np.array(test_df[my_var]),bins = np.linspace(test_df[my_var].min(),test_df[my_var].max(),101))[0]
    #test_hist = test_hist/test_hist.sum(axis=0, keepdims=True) 
    #test_perc = np.cumsum(test_hist)
    test_mean = test_df[my_var].mean()
    test_std = test_df[my_var].std()
    test_perc = np.nanpercentile(test_df[my_var],my_perc)
    
    if normalize_y:
      test_perc = [(my_test_perc-test_mean)/test_std for my_test_perc in test_perc]  
    plt.plot(gaussian_perc,test_perc,label = my_test,color = my_color)

  #ax.set_xticks(np.round(np.linspace(np.nanmin(x), np.nanmax(x), 5), 5))
  #ax.set_yticks(np.round(np.linspace(vmin, vmax, 5), 5))
  vmin = np.nanmin([gnu_perc,intel_perc,test_perc])
  vmax = np.nanmax([gnu_perc,intel_perc,test_perc])
  ax.set_xticks((np.linspace(np.nanmin(x), np.nanmax(x), 5)))
  ax.set_yticks((np.linspace(vmin, vmax, 5)))


  plt.title(my_var)
  plt.legend()
  return ax #fig

gaussian_qqplot('BURDEN2',False)


In [ ]:

from scipy.stats import norm
normalize_y = False

# QUantile ratio plot
def gaussian_qrplot(my_var,normalize_y):
  fig = plt.figure(2,figsize = (8,8))
  my_perc = np.arange(1,100,1)
  # Set to True to use only 25 simulations 
  matched = False 

  if not matched:
    gnu_df = df[(df.Label == 'GNU')]
    intel_df = df[(df.Label == 'Intel') ]
    intel_perc = np.nanpercentile(df[(df.Label == 'Intel')][my_var],my_perc)
  else:
    match_list = [str(simu).zfill(3) for simu in range(25)]
    #match_list.remove('006')
    gnu_df = df[(df.Label == 'GNU') & (df.Simulation.isin(match_list))]
    intel_df = df[(df.Label == 'Intel') & (df.Simulation.isin(match_list))]
  gnu_mean = gnu_df[my_var].mean()
  gnu_std = gnu_df[my_var].std()
  gnu_perc = np.nanpercentile(df[df.Label == 'GNU'][my_var],my_perc)
  if normalize_y:
    gnu_perc = [(my_gnu_perc-gnu_mean)/gnu_std for my_gnu_perc in gnu_perc]
  intel_mean = intel_df[my_var].mean()
  intel_std = intel_df[my_var].std()
  intel_perc = np.nanpercentile(df[(df.Label == 'Intel')][my_var],my_perc)
  if normalize_y:
    intel_perc = [(my_intel_perc-intel_mean)/intel_std for my_intel_perc in intel_perc]
  test_perc = np.nanpercentile(df[(df.Label == 'Test')][my_var],my_perc)

  new_qr = intel_perc/gnu_perc
  #gnu_hist = np.histogram(np.array(gnu_df[my_var]),bins = np.linspace(gnu_df[my_var].min(),gnu_df[my_var].max(),101))[0]
  #gnu_hist = gnu_hist/gnu_hist.sum(axis=0, keepdims=True) 
  #gnu_perc = np.cumsum(gnu_hist)
  #intel_hist = np.histogram(np.array(intel_df[my_var]),bins = np.linspace(intel_df[my_var].min(),intel_df[my_var].max(),101))[0]
  #intel_hist = intel_hist/intel_hist.sum(axis=0, keepdims=True) 
  #intel_perc = np.cumsum(intel_hist)

  x = [norm.ppf(my_quant) for my_quant in np.linspace(0.01,0.99,99)]
  #gaussian_perc = norm.cdf(x)
  gaussian_perc = x.copy()

  vmin = np.nanmin(new_qr)
  vmax = np.nanmax(new_qr)
  plt.plot([np.nanmin(x),np.nanmax(x)],[vmin,vmax],'k--',alpha = 0.5)#,label = 'Gaussian'

  plt.plot(gaussian_perc,new_qr,'k',linestyle = '-',label = 'Quant-ratio',zorder = 10)
  #plt.plot(gaussian_perc,intel_perc,'k-',label = 'Intel',zorder = 10)


  plt.xlabel('Standard Normal Quantiles',fontsize = 12)
  plt.ylabel('Quantile Ratio Intel/GNU',fontsize = 12)

  test_list = np.unique(df[df.Label == 'Test']['Simulation'])
  for my_test,my_color in zip(test_list,color_list):
    test_df = df[df.Simulation == my_test]
    #test_hist = np.histogram(np.array(test_df[my_var]),bins = np.linspace(test_df[my_var].min(),test_df[my_var].max(),101))[0]
    #test_hist = test_hist/test_hist.sum(axis=0, keepdims=True) 
    #test_perc = np.cumsum(test_hist)
    test_mean = test_df[my_var].mean()
    test_std = test_df[my_var].std()
    test_perc = np.nanpercentile(test_df[my_var],my_perc)
    
    if normalize_y:
      test_perc = [(my_test_perc-test_mean)/test_std for my_test_perc in test_perc]  
    #plt.plot(gaussian_perc,test_perc,label = my_test,color = my_color)

  #ax.set_xticks(np.round(np.linspace(np.nanmin(x), np.nanmax(x), 5), 5))
  #ax.set_yticks(np.round(np.linspace(vmin, vmax, 5), 5))
  vmin = np.nanmin([gnu_perc,intel_perc,test_perc])
  vmax = np.nanmax([gnu_perc,intel_perc,test_perc])
  ax.set_xticks((np.linspace(np.nanmin(x), np.nanmax(x), 5)))
  ax.set_yticks((np.linspace(vmin, vmax, 5)))


  plt.title(my_var)
  plt.legend()
  return ax #fig

#gaussian_qrplot('BURDEN2',False)
### 9 subplots
normalize_y = False
nrow = 3
ncol = 3
fig = plt.figure(figsize=(12,12))
plt.subplots_adjust(wspace = 0.3,hspace= 0.3)

#fig = plt.figure(figsize=(8,8))
var_list = ['TS','TSMN','TSMX','VT','VU','U10','LWCF','BURDEN2','CCN3']

plt.tight_layout()
my_perc = np.arange(1,100,1)
# Set to True to use only 25 simulations 
matched = False 
use_year = 100
nyear= 100

if not matched:
  gnu_df = df[(df.Label == 'GNU') & (df.nyear >= nyear - use_year)]
  intel_df = df[(df.Label == 'Intel') & (df.nyear >= nyear - use_year)]
  intel_perc = np.nanpercentile(intel_df[my_var],my_perc)
  all_test_df= df[(df.Label == 'Test') & (df.nyear >= nyear - use_year)]
else:
  match_list = [str(simu).zfill(3) for simu in range(25)]
  match_list.remove('000')
  gnu_df = df[(df.Label == 'GNU') & (df.Simulation.isin(match_list)) & (df.nyear >= nyear - use_year)]
  intel_df = df[(df.Label == 'Intel') & (df.Simulation.isin(match_list)) & (df.nyear >= nyear - use_year)]
  all_test_df= df[(df.Label == 'Test') & (df.nyear >= nyear - use_year)]

for irow in range(nrow):
  for icol in range(ncol):
    idx = ncol*irow+icol
    ax = plt.subplot(nrow,ncol,idx+1)
    my_var = var_list[idx]

      

    gnu_mean = gnu_df[my_var].mean()
    gnu_std = gnu_df[my_var].std()
    #if np.nanmin(gnu_df[my_var])*np.nanmax(gnu_df[my_var]) < 0:
    gnu_perc = np.nanpercentile(abs(gnu_df[my_var]),my_perc)
    if normalize_y:
      gnu_perc = [(my_gnu_perc-gnu_mean)/gnu_std for my_gnu_perc in gnu_perc]
    intel_mean = intel_df[my_var].mean()
    intel_std = intel_df[my_var].std()
    #intel_perc = np.nanpercentile(intel_df[my_var],my_perc)
    intel_perc = np.nanpercentile(abs(intel_df[my_var]),my_perc)
    if normalize_y:
      intel_perc = [(my_intel_perc-intel_mean)/intel_std for my_intel_perc in intel_perc]
    test_perc = np.nanpercentile(df[(df.Label == 'Test')][my_var],my_perc)

    #gnu_hist = np.histogram(np.array(gnu_df[my_var]),bins = np.linspace(gnu_df[my_var].min(),gnu_df[my_var].max(),101))[0]
    #gnu_hist = gnu_hist/gnu_hist.sum(axis=0, keepdims=True) 
    #gnu_perc = np.cumsum(gnu_hist)
    #intel_hist = np.histogram(np.array(intel_df[my_var]),bins = np.linspace(intel_df[my_var].min(),intel_df[my_var].max(),101))[0]
    #intel_hist = intel_hist/intel_hist.sum(axis=0, keepdims=True) 
    #intel_perc = np.cumsum(intel_hist)

    x = [norm.ppf(my_quant) for my_quant in np.linspace(0.01,0.99,99)]
    #gaussian_perc = norm.cdf(x)
    gaussian_perc = x.copy()
    new_qr  = intel_perc/gnu_perc
    vmin = np.nanmin(new_qr)
    vmax = np.nanmax(new_qr)
    

    #plt.plot([np.nanmin(x),np.nanmax(x)],[vmin,vmax],'k--',alpha = 0.5)#,label = 'Gaussian'
    plt.hlines(1,np.nanmin(x),np.nanmax(x),linestyle ='--',alpha = 0.5)#,label = 'Gaussian'

    plt.plot(gaussian_perc,new_qr,'k',linestyle = '-',label = 'Quant-Ratio',zorder = 10)
    #plt.plot(gaussian_perc,intel_perc,'k-',label = 'Intel',zorder = 10)

    if idx == 7:
      plt.xlabel('Standard Normal Quantiles',fontsize = 12)
    if idx == 3:
      plt.ylabel('Quantile Ratio Intel/GNU',fontsize = 12)


    test_list = np.unique(df[df.Label == 'Test']['Simulation'])
    for my_test,my_color in zip(test_list,color_list):
      test_df = df[df.Simulation == my_test]
      #test_hist = np.histogram(np.array(test_df[my_var]),bins = np.linspace(test_df[my_var].min(),test_df[my_var].max(),101))[0]
      #test_hist = test_hist/test_hist.sum(axis=0, keepdims=True) 
      #test_perc = np.cumsum(test_hist)
      test_mean = test_df[my_var].mean()
      test_std = test_df[my_var].std()
      test_perc = np.nanpercentile(test_df[my_var],my_perc)
      if normalize_y:
        test_perc = [(my_test_perc-test_mean)/test_std for my_test_perc in test_perc]  
      #plt.plot(gaussian_perc,test_perc,label = my_test,color = my_color)


    #vmin = np.nanmin([gnu_perc,intel_perc,test_perc])
    #vmax = np.nanmax([gnu_perc,intel_perc,test_perc])
    #vmin = np.nanmin([gnu_perc,intel_perc])
    #vmax = np.nanmax([gnu_perc,intel_perc])
    ax.set_xticks((np.linspace(np.nanmin(x), np.nanmax(x), 5)))
    ax.set_yticks((np.linspace(vmin, vmax, 5)))


    plt.title(my_var)
    if idx == 2:
      plt.legend(frameon=False,bbox_to_anchor=(1.1, 0.5, 0.5, 0.5))
    

#plt.savefig(local_dir+'qq_plot_nine_variables_latter_'+str(use_year)+'.png',dpi = 500)


In [ ]:
### 9 subplots
import matplotlib.ticker as ticker

normalize_y = False
nrow = 3
ncol = 3
fig = plt.figure(figsize=(12,12))
plt.subplots_adjust(wspace = 0.3,hspace= 0.3)

#fig = plt.figure(figsize=(8,8))
var_list = ['TS','TSMN','TSMX','VT','VU','U10','LWCF','BURDEN2','CCN3']

plt.tight_layout()
my_perc = np.arange(1,100,1)
# Set to True to use only 25 simulations 
matched = False 
use_year = 90
nyear= 100

if not matched:
  gnu_df = df[(df.Label == 'GNU') & (df.nyear >= nyear - use_year)]
  intel_df = df[(df.Label == 'Intel') & (df.nyear >= nyear - use_year)]
  intel_perc = np.nanpercentile(intel_df[my_var],my_perc)
  all_test_df= df[(df.Label == 'Test') & (df.nyear >= nyear - use_year)]
else:
  match_list = [str(simu).zfill(3) for simu in range(25)]
  match_list.remove('000')
  gnu_df = df[(df.Label == 'GNU') & (df.Simulation.isin(match_list)) & (df.nyear >= nyear - use_year)]
  intel_df = df[(df.Label == 'Intel') & (df.Simulation.isin(match_list)) & (df.nyear >= nyear - use_year)]
  all_test_df= df[(df.Label == 'Test') & (df.nyear >= nyear - use_year)]

for irow in range(nrow):
  for icol in range(ncol):
    idx = ncol*irow+icol
    ax = plt.subplot(nrow,ncol,idx+1)
    my_var = var_list[idx]

      

    gnu_mean = gnu_df[my_var].mean()
    gnu_std = gnu_df[my_var].std()
    gnu_perc = np.nanpercentile(gnu_df[my_var],my_perc)
    if normalize_y:
      gnu_perc = [(my_gnu_perc-gnu_mean)/gnu_std for my_gnu_perc in gnu_perc]
    intel_mean = intel_df[my_var].mean()
    intel_std = intel_df[my_var].std()
    intel_perc = np.nanpercentile(intel_df[my_var],my_perc)
    if normalize_y:
      intel_perc = [(my_intel_perc-intel_mean)/intel_std for my_intel_perc in intel_perc]
    test_perc = np.nanpercentile(df[(df.Label == 'Test')][my_var],my_perc)

    #gnu_hist = np.histogram(np.array(gnu_df[my_var]),bins = np.linspace(gnu_df[my_var].min(),gnu_df[my_var].max(),101))[0]
    #gnu_hist = gnu_hist/gnu_hist.sum(axis=0, keepdims=True) 
    #gnu_perc = np.cumsum(gnu_hist)
    #intel_hist = np.histogram(np.array(intel_df[my_var]),bins = np.linspace(intel_df[my_var].min(),intel_df[my_var].max(),101))[0]
    #intel_hist = intel_hist/intel_hist.sum(axis=0, keepdims=True) 
    #intel_perc = np.cumsum(intel_hist)

    x = [norm.ppf(my_quant) for my_quant in np.linspace(0.01,0.99,99)]
    #gaussian_perc = norm.cdf(x)
    gaussian_perc = x.copy()
    slope, intercept, r_value, p_value, std_err = stats.linregress(x,intel_perc)
    print('Linear fit of Intel '+str(my_var)+' r-value =', r_value)
    #print('Linear fit of Intel '+str(my_var)+' p-value =', p_value)

    vmin = np.nanmin([gnu_perc,intel_perc])
    vmax = np.nanmax([gnu_perc,intel_perc])
    

    plt.plot([np.nanmin(x),np.nanmax(x)],[vmin,vmax],'k--',alpha = 0.5)#,label = 'Gaussian'

    plt.plot(gaussian_perc,gnu_perc,'k',linestyle = 'dashdot',label = 'GNU',zorder = 10)
    plt.plot(gaussian_perc,intel_perc,'k-',label = 'Intel',zorder = 10)

    if idx == 7:
      plt.xlabel('Standard Normal Quantiles',fontsize = 12)
    if idx == 3:
      plt.ylabel('Intel/GNU/Test Quantiles',fontsize = 12)


    test_list = np.unique(df[df.Label == 'Test']['Simulation'])
    for my_test,my_color in zip(test_list,color_list):
      test_df = df[df.Simulation == my_test]
      #test_hist = np.histogram(np.array(test_df[my_var]),bins = np.linspace(test_df[my_var].min(),test_df[my_var].max(),101))[0]
      #test_hist = test_hist/test_hist.sum(axis=0, keepdims=True) 
      #test_perc = np.cumsum(test_hist)
      test_mean = test_df[my_var].mean()
      test_std = test_df[my_var].std()
      test_perc = np.nanpercentile(test_df[my_var],my_perc)
      if normalize_y:
        test_perc = [(my_test_perc-test_mean)/test_std for my_test_perc in test_perc]  
      #plt.plot(gaussian_perc,test_perc,label = my_test,color = my_color)


    #vmin = np.nanmin([gnu_perc,intel_perc,test_perc])
    #vmax = np.nanmax([gnu_perc,intel_perc,test_perc])
    vmin = np.nanmin([gnu_perc,intel_perc])
    vmax = np.nanmax([gnu_perc,intel_perc])
    ax.set_xticks((np.linspace(np.nanmin(x), np.nanmax(x), 5)))
    ax.set_yticks((np.linspace(vmin, vmax, 5)))


    plt.title(my_var)
    if idx == 2:
      plt.legend(frameon=False)#bbox_to_anchor=(1.1, 0.5, 0.5, 0.5)
    if idx == 7:
      ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%0.2e'))
    else:
      ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%0.2f'))


plt.savefig(local_dir+'qq_plot_nine_variables_latter_'+str(use_year)+'.png',dpi = 500)


In [ ]:

for my_var in removed_column_list:
  r_value_list = []
  for my_test in test_list:

    gnu_mean = gnu_df[my_var].mean()
    gnu_std = gnu_df[my_var].std()
    gnu_perc = np.nanpercentile(gnu_df[my_var],my_perc)
    if normalize_y:
      gnu_perc = [(my_gnu_perc-gnu_mean)/gnu_std for my_gnu_perc in gnu_perc]
    intel_mean = intel_df[my_var].mean()
    intel_std = intel_df[my_var].std()
    intel_perc = np.nanpercentile(intel_df[my_var],my_perc)
    if normalize_y:
      intel_perc = [(my_intel_perc-intel_mean)/intel_std for my_intel_perc in intel_perc]
    test_perc = np.nanpercentile(df[(df.Label == 'Test') & (df.Simulation == my_test)][my_var],my_perc)


    x = [norm.ppf(my_quant) for my_quant in np.linspace(0.01,0.99,99)]
    #gaussian_perc = norm.cdf(x)
    gaussian_perc = x.copy()
    slope, intercept, r_value, p_value, std_err = stats.linregress(x,test_perc)
    if r_value < 0.98:
      print('Linear fit of '+str(my_test)+' '+str(my_var)+' r-value =', r_value)
    if slope > 1.1:
      print('Linear fit of '+str(my_test)+' '+str(my_var)+' slope =', slope)

    r_value_list.append(r_value)


In [ ]:
np.nanmin(r_value_list)

This is the quantile ratio plot against standard Gaussian. If the points align on a straight line, then it is Gaussian. The steeper the slope is, the wider. the distribution. 
If it is flat in lower x, it means it misses the low extreme. 

In [ ]:
var_list = column_list

@interact(my_var=var_list)
def make_plot_for(my_var='TS'):
    plot = gaussian_qqplot(my_var,normalize_y = False)
    #display(plot)

In [ ]:
#df[(df.Label == 'Intel') & (df.Simulation == '115') & (df.nyear > 94)].describe()
fs = 14

#### Show all standard deviation

There should not be a standard deviation exceeding E+30

In [ ]:
#df[(df.Label == 'Intel') & (df.Simulation == '034')]['TS'].plot()
'''
for col in column_list:
  for my_simu in ['Intel','GNU','Test']:
    #if my_simu == 'Intel':
    #  my_std = df[(df.Label == 'Intel') & (df.Simulation != '006')][[col]].std()
    #else:
    #  my_std = df[(df.Label == my_simu)][[col]].std()
    my_std = df[(df.Label == my_simu)][[col]].std()
    if my_std[col] > 100:
      print(my_simu,col,my_std[col])
'''

The first years could be different

In [ ]:
'''
divide_year = 9
my_var = 'TS'
intel_mean = intel_df[intel_df.nyear > divide_year][my_var].mean()
gnu_mean = gnu_df[gnu_df.nyear > divide_year][my_var].mean()
my_cov = np.zeros(divide_year)
for first_years in range(divide_year):
  for simu in match_list:
    xi = intel_df[(intel_df.Simulation == simu) & (intel_df.nyear == first_years)][my_var].mean()
    yi = gnu_df[(gnu_df.Simulation == simu) & (gnu_df.nyear == first_years)][my_var].mean()
    my_cov[first_years] = my_cov[first_years] + (xi - intel_mean)*(yi - gnu_mean)
my_cov
'''

In [ ]:
#sns.lineplot(range(divide_year),my_cov)
#plt.hlines(0,0,divide_year,linestyles='--')

#### Testing normality

Here we use KS-test or Shapiro-Wilk test to test normality of each variable, all years & simulations aggregated. 

In [ ]:

my_ens = df[df.Simulation == test_list[5]]#intel_df[intel_df.Simulation != '000']
ci =0.05
#my_ens = intel_df
shap_pval = np.zeros(len(column_list))
ks_pval = np.zeros(len(column_list))
shap_pval.fill(np.nan)
ks_pval.fill(np.nan)
for idx,my_var in enumerate(column_list):
  if my_ens[my_var].std() > 0:
    shap_pval[idx] = stats.shapiro((my_ens[my_var] - my_ens[my_var].mean())/(my_ens[my_var].std()))[1]
    ks_pval[idx] = stats.kstest((my_ens[my_var] - my_ens[my_var].mean())/(my_ens[my_var].std()),'norm')[1]
    #ks_pval[idx] = stats.kstest((my_ens[my_var] - my_ens[my_var].mean()),'norm')[1]
  if ks_pval[idx] < ci:
    print(my_var)
'''
var_list = ['TS','TSMN','TSMX','VT','VU','U10','LWCF','BURDEN2','CCN3']
my_ens = intel_df[intel_df.Simulation != '000']
shap_pval = np.zeros(len(var_list))
ks_pval = np.zeros(len(var_list))
shap_pval.fill(np.nan)
ks_pval.fill(np.nan)
my_mean =  np.zeros(len(var_list))
my_variance =  np.zeros(len(var_list))
for idx,my_var in enumerate(var_list):
  if my_ens[my_var].std() > 0:
    shap_pval[idx] = stats.shapiro((my_ens[my_var] - my_ens[my_var].mean())/(my_ens[my_var].std()))[1]
    ks_pval[idx] = stats.kstest((my_ens[my_var] - my_ens[my_var].mean())*1./(my_ens[my_var].std()),'norm')[1]
    my_mean[idx] = my_ens[my_var].mean()
    my_variance[idx] = my_ens[my_var].var()
'''

In [ ]:
column_list

In [ ]:
fs = 14

In [ ]:
mean_stats = pd.DataFrame()
mean_stats['Variable'] = var_list
#mean_stats['Mean'] = my_mean
#mean_stats['Variance'] = my_variance

mean_stats['KS-pvalue'] = [round(ks_p,2) for ks_p in ks_pval]
#my_ens['BURDEN2'].hist()
mean_stats.transpose()


#df[df['TSMX'] < 290]

In [ ]:
plt.plot(np.sort(shap_pval),label='Shapiro-Wilk test')
plt.plot(np.sort(ks_pval),label = 'KS test')
plt.legend()

len(ks_pval[ks_pval < 0.05])/len(ks_pval)

### Testing hypothesis

In [ ]:
def KL(P,Q):
     epsilon = 0.00001

     # You may want to instead make copies to avoid changing the np arrays.
     P = P+epsilon
     Q = Q+epsilon

     divergence = np.sum(P*np.log(P/Q))
     return divergence

#### KS-test

Test the null hypothesis Ho:
The randomly picked GNU run is different from the Intel ensemble distribution across 100 years (assuming stationarity). 

If the K-S statistic is small or the p-value is high, then we cannot reject the hypothesis that the distributions of the two samples are the same.
https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.ks_2samp.html

In [ ]:
'''
### Testing a random GNU run
use_year = False
my_var = 'TS'
matched = True 


if use_year:
  match_list = [str(simu).zfill(3) for simu in np.random.randint(0,25,1)]
  intel_df = df[(df.Label == 'Intel') & (df.Simulation.isin(match_list))]
  gnu_df = df[(df.Label == 'GNU') & (df.Simulation.isin(match_list))]
else:
  if not matched:
    gnu_df = df[(df.Label == 'GNU')]
    intel_df = df[(df.Label == 'Intel') ]
  else:
    match_list = [str(simu).zfill(3) for simu in range(25)]
    #match_list.remove('006')
    gnu_df = df[(df.Label == 'GNU') & (df.Simulation.isin(match_list))]
    intel_df = df[(df.Label == 'Intel') & (df.Simulation.isin(match_list))]

res = stats.ks_2samp(np.array(gnu_df[my_var]),np.array(intel_df[my_var]))
'''

In [ ]:
# All matched ensembles
use_year = 100
matched = False
match_list = [str(simu).zfill(3) for simu in range(25)]
match_list.remove('000')
### If use 25 runs


if matched:
  intel_df = df[(df.Label == 'Intel') & (df.Simulation.isin(match_list)) & (df.nyear >= nyear - use_year)]
  gnu_df = df[(df.Label == 'GNU') & (df.Simulation.isin(match_list)) & (df.nyear >= nyear - use_year)]
else:
  intel_df = df[(df.Label == 'Intel') & (df.nyear >= nyear - use_year)]
  gnu_df = df[(df.Label == 'GNU') & (df.nyear >= nyear - use_year)]
  print('Latter '+str(use_year)+' years are used.')


var_list = column_list #['TS','TSMN','TSMX']#
ci = 0.05 # confidence interval
ks_pvalue = np.zeros(len(var_list))
ks_stats = np.zeros(len(var_list))
for icol,my_var in enumerate(var_list):
  res = stats.ks_2samp(np.array(gnu_df[my_var]),np.array(intel_df[my_var]))
  ks_stats[icol]= res[0]
  ks_pvalue[icol]= res[1]
  if res[1] < ci:
    print('This is variable: ',my_var)

fig = plt.figure(figsize = (12,5))
'''
plt.scatter(ks_stats,ks_pvalue)
xlim = plt.gca().get_xlim()
plt.hlines(ci,xlim[0],xlim[1],linestyle = '--')
print('Percent of variables rejected null hypothesis at CI=',str(int(ci*100)),'% is ',str(len(ks_pvalue[np.where(ks_pvalue < ci)])*1./len(ks_pvalue)))
'''
ax = plt.subplot(1,2,1)
plt.hist(ks_pvalue,bins = np.arange(0,1,0.05),histtype='stepfilled',alpha = 0.6, linewidth=2)
plt.ylabel('Histogram',fontsize =fs)
plt.xlim(-0.05,1.05)
plt.vlines(0.05,0,13,linestyle = '--')
plt.annotate('95% CI',xy = (0.05,10),xytext=(0.1,11),arrowprops=dict(arrowstyle="->",
                            connectionstyle="arc3"))
plt.text(1.0,-1.5,'KS test p-values',fontsize = fs)
ax = plt.subplot(1,2,2)
plt.hist(ks_pvalue,bins = np.arange(0,1,0.05),density=True, cumulative = True,label='CDF', 
         histtype='step', alpha=0.55, color='k',linewidth= 2)
plt.ylabel ('CDF',fontsize =fs)

In [ ]:
fig = plt.figure(3,figsize = (8,6))
bins = np.linspace(np.nanmin([intel_df[my_var].min(),gnu_df[my_var].min()]),np.nanmax([intel_df[my_var].max(),gnu_df[my_var].max()]),40)
plt.hist(intel_df[my_var], bins=bins, density=True, histtype='step', cumulative=True,
        label='Intel',lw=2)
plt.hist(gnu_df[my_var], bins=bins, density=True, histtype='step', cumulative=True,
        label='GNU',lw=2)
plt.legend(loc = 2,frameon=False)
print('KS-statistics: ',res)


In [ ]:
fig = plt.figure(3,figsize = (8,6))
test_df = df[df.Simulation == 'rh-min-low']
my_var = 'TS'

bins = np.linspace(np.nanmin([intel_df[my_var].min(),test_df[my_var].min()]),np.nanmax([intel_df[my_var].max(),test_df[my_var].max()]),40)
hist1= plt.hist(intel_df[my_var], bins=bins, density=True, histtype='step', cumulative=True, label='Intel',lw=2)
hist2 =plt.hist(test_df[my_var], bins=bins, density=True, histtype='step', cumulative=True, label='rh-min-low',lw=2)
plt.legend(loc = 2,frameon=False)
plt.ylabel('CDF',fontsize =fs)
plt.xlabel('TS (K)',fontsize =fs)
print('KS-statistics: ',res)
dcdf = abs(hist1[0] - hist2[0])
mask = np.where(dcdf == np.nanmax(dcdf))[0]
plt.vlines(bins[mask]+0.5*(bins[1]-bins[0]),hist1[0][mask],hist2[0][mask])


In [ ]:
# For each of 25 paired runs

var_list = column_list #['TS','TSMN','TSMX']#
ci = 0.05 # confidence interval
ks_pvalue = np.zeros((len(var_list),len(match_list)))
ks_stats = np.zeros((len(var_list),len(match_list)))
for icol,my_var in enumerate(var_list):

  for isimu,my_simu in enumerate(match_list):
    res = stats.ks_2samp(np.array(gnu_df[gnu_df.Simulation == my_simu][my_var]),np.array(intel_df[intel_df.Simulation == my_simu][my_var]))
    ks_stats[icol,isimu]= res[0]
    ks_pvalue[icol,isimu]= res[1]
    if res[1] < ci:
       print('This is variable: ',my_var,' ',my_simu)


In [ ]:
# For each of test runs against Intel ensemble

var_list = column_list #['TS','TSMN','TSMX']#
ci = 0.05 # confidence interval
num_rej = np.zeros((len(var_list),len(test_list)))
intel_df = df[(df.Label == 'Intel') & (df.nyear >= nyear - use_year)]
for icol,my_var in enumerate(var_list):
  for itest,my_test in enumerate(test_list):
    test_df = df[(df.Simulation == my_test) & (df.nyear >= nyear - use_year)]
    res = stats.ks_2samp(np.array(test_df[my_var]),np.array(intel_df[my_var]))
    if res[1] < ci:
      print('This is variable: ',my_var,' ',my_test)
      num_rej[icol,itest]+=1 
    

In [ ]:
tot_rej_var = np.sum(num_rej[:,:],axis=1)
for idx, my_var in enumerate(removed_column_list):
  if tot_rej_var[idx] == 5:
    print(my_var,tot_rej_var[idx])

In [ ]:
len(removed_column_list)

In [ ]:
### For each variable, how many simulations are rejected
fig = plt.figure(figsize = (8,8))
plt.hist(np.sum(ks_pvalue[:] < ci,axis=0))
xlim = plt.gca().get_xlim()
plt.hlines(ci,xlim[0],xlim[1],linestyle = '--')
print('Percent of variables rejected null hypothesis at CI=',str(int(ci*100)),'% is ',str(len(ks_pvalue[np.where(ks_pvalue < ci)])*1./len(ks_pvalue)))

In [ ]:
### For each simulation, what percentage of variables are rejected
'''
fig = plt.figure(figsize = (12,5))
ax = plt.subplot(1,2,1)
bins = np.arange(-0.5,17.5,1)
nbins = len(bins)
plt.hist(np.sum(ks_pvalue[:] < ci,axis=1),bins = bins)
xlim = plt.gca().get_xlim()
plt.ylabel('Histogram',fontsize = fs)
plt.text(12,-7,'# of simulations rejected at 95% CI',fontsize = fs)
plt.xticks(labels=[str(int(x_tick+0.5)) for x_tick in bins[0::4]],ticks= (np.arange(nbins+1) )[0::4])

#plt.hlines(ci,0,16,linestyle = '--')
print('Percent of variables rejected null hypothesis at CI=',str(int(ci*100)),'% is ',str(len(ks_pvalue[np.where(ks_pvalue < ci)])*1./len(ks_pvalue)))
plt.annotate(column_list[np.sum(ks_pvalue[:] < ci,axis=1) > 10.][0],xy =(15,1), xytext=(12,4),arrowprops=dict(arrowstyle="->",
                            connectionstyle="arc3"))

ax = plt.subplot(1,2,2)
plt.hist(np.sum(ks_pvalue[:] < ci,axis=1), bins= np.arange(-0.5,17.5,1), density=True, cumulative=True, label='CDF DATA', 
         histtype='step', alpha=0.55, color='k',linewidth= 2)
plt.ylabel('CDF',fontsize=fs)
plt.xticks(labels=[str(int(x_tick+0.5)) for x_tick in bins[0::4]],ticks= (np.arange(nbins+1) )[0::4])
'''

In [ ]:
bins

In [ ]:
det = np.sum(ks_pvalue[:] < 0.05,axis=1)
for idet,my_det in enumerate(det):
  if my_det > 2:
    print(my_det,column_list[idet])
    

In [ ]:
my_df = gnu_df.copy()
divide_year = 5+nyear - use_year
test_df = my_df[(my_df.nyear <= divide_year)]
ens_df = my_df[(my_df.nyear > divide_year)]


x = np.array(ens_df[my_var])
y = np.array(test_df[my_var])
bins = np.linspace(np.nanmin(np.hstack((x,y))),np.nanmax(np.hstack((x,y))),40)
 
p = plt.hist(x,bins=bins)[0]
q = plt.hist(y,bins=bins)[0]
p = p/p.sum(axis=0, keepdims=True) 
q = q/q.sum(axis=0, keepdims=True) 
#z = np.array(test_df[my_var])



# Note slight difference in the final result compared to Dawny33
print('This is GNU.000')
res = stats.ks_2samp(x,y)
print('KS-statistics: ',res)
#kl_res = scipy.special.kl_div(x,y)
kl_res = KL(p,q)
Sp = stats.entropy(np.array(p))
Sq = stats.entropy(np.array(q))
print('KL-entropy of A: ',Sp)
print('KL-entropy of B: ',Sq)
print('KL-divergence: ',kl_res)

sns.distplot(x,bins=bins)
sns.distplot(y,bins=bins)

In [ ]:
'''
import plotly.graph_objects as go
import plotly.figure_factory as ff

enable_plotly_in_cell()

# Load data
df = intel_df.copy()
#df.columns = [col.replace("AAPL.", "") for col in df.columns]

# Create figure
fig = go.Figure()

fig.add_trace(
    ff.create_distplot(df.TS, 'Intel'))

# Set title
fig.update_layout(
    title_text="Time series with range slider and selectors"
)

# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()
'''

In [ ]:
### Test all test runs
summary_stat = pd.DataFrame()
one_line = {
    'Dist_A':'',
    'Dist_B':'',
    'KS-stats':0,
    'KS-pvalue':0,
    'KL-div':0,
    'KL-entropy':0
}
test_list
use_year = 90
my_var = 'TS'
run_list = np.unique(intel_df.Simulation)

for my_test in test_list:
  new_line = one_line.copy()
  test_df = df[(df.Simulation == my_test) & (df.nyear >= nyear - use_year)]
  x = np.array(intel_df[my_var])
  y = np.array(test_df[my_var])
  bins = np.linspace(np.nanmin(np.hstack((x,y))),np.nanmax(np.hstack((x,y))),40)



  p = plt.hist(x,bins=bins)[0]
  q = plt.hist(y,bins=bins)[0]
  p = p/p.sum(axis=0, keepdims=True) 
  q = q/q.sum(axis=0, keepdims=True) 

  print('This is test run: ',my_test)
  res = stats.ks_2samp(x,y)
  print('KS-statistics: ',res)
  Sp = stats.entropy(np.array(x))
  print('KL-entropy of A: ',Sp)
  #kl_res = scipy.special.kl_div(p,q)
  kl_res = KL(p,q)
  print('KL-divergence: ',kl_res)
  new_line['Dist_A'] = my_test
  new_line['Dist_B'] = str(len(run_list))+' Intel'
  new_line['KS-stats'] = round(res[0],2)
  new_line['KS-pvalue'] = round(res[1],2)
  new_line['KL-entropy'] = round(Sp,2)
  new_line['KL-div'] = round(kl_res,2)

  summary_stat = summary_stat.append(new_line,ignore_index = True)

### Add GNU 000 for referencfe
test_df = df[(df.Label == 'Intel') & (df.Simulation == '001')]

x = np.array(gnu_df[my_var])
y = np.array(test_df[my_var])
bins = np.linspace(np.nanmin(np.hstack((x,y))),np.nanmax(np.hstack((x,y))),40)
 
p = plt.hist(x,bins=bins)[0]
q = plt.hist(y,bins=bins)[0]
p = p/p.sum(axis=0, keepdims=True) 
q = q/q.sum(axis=0, keepdims=True) 
#z = np.array(test_df[my_var])



# Note slight difference in the final result compared to Dawny33
print(KL(p, q)) # 0.775278939433
print('This is GNU.000')
res = stats.ks_2samp(x,y)
print('KS-statistics: ',res)
#kl_res = scipy.special.kl_div(x,y)
kl_res = KL(p,q)
Sp = stats.entropy(np.array(x))
print('KL-entropy of A: ',Sp)
print('KL-divergence: ',kl_res)

new_line = one_line.copy()
new_line['Dist_A'] = 'GNU.000'
#new_line['Dist_B'] = str(len(run_list))+' GNU'
new_line['Dist_B'] = str(len(run_list))+' Intel'
new_line['KS-stats'] = round(res[0],2)
new_line['KS-pvalue'] = round(res[1],2)
new_line['KL-entropy'] = round(Sp,2)
new_line['KL-div'] = round(kl_res,2)
summary_stat = summary_stat.append(new_line,ignore_index = True)


In [ ]:
#summary_stat = summary_stat.drop(axis=1,columns=['KL-entropy'])
summary_stat[['Dist_A','Dist_B','KS-stats','KS-pvalue','KL-div']].transpose()


In [ ]:
ens_df = df[(df.Label == 'Intel') & (df.nyear >= nyear - use_year)]
add_test_df =  df[(df.Label == 'Intel') & (df.Simulation == '001')  & (df.nyear >= nyear - use_year)]
my_var='TS'
alpha = 0.05

def stats_table(my_var,ens_df,add_test_df):
  # my_var is for the variable to show
  # add_test_df is the one member to add to comparison (in addition to all Test Simulations)
  # ens_df is the ensemble to test against (Dist B)
  res = stats.ks_2samp(gnu_df[my_var],intel_df[my_var])
  ### Test all test runs
  summary_stat = pd.DataFrame()
  one_line = {
      'Dist_A':'',
      'Dist_B':'',
      'KS-stats':0,
      'KS-pvalue':0,
      'KL-div':0,
      'KL-entropy':0
  }
  test_list
  run_list = np.unique(ens_df.Simulation)

  for my_test in test_list:
    new_line = one_line.copy()
    test_df = df[df.Simulation == my_test]
    # Change to Intel to test against Intel Ensemble
    x = np.array(ens_df[my_var])
    y = np.array(test_df[my_var])
    bins = np.linspace(np.nanmin(np.hstack((x,y))),np.nanmax(np.hstack((x,y))),40)


    p = np.histogram(x,bins = bins)[0]
    q = np.histogram(y,bins = bins)[0]
    #p = plt.hist(x,bins=bins)[0]
    #q = plt.hist(y,bins=bins)[0]
    p = p/p.sum(axis=0, keepdims=True) 
    q = q/q.sum(axis=0, keepdims=True) 

    #print('This is test run: ',my_test)
    res = stats.ks_2samp(x,y)
    #print('KS-statistics: ',res)
    Sp = stats.entropy(np.array(x))
    #print('KL-entropy of A: ',Sp)
    kl_res = KL(p,q)
    #print('KL-divergence: ',kl_res)
    new_line['Dist_A'] = my_test
    new_line['Dist_B'] = str(len(run_list))+' '+str(ens_df.Label.iloc[0])
    new_line['KS-stats'] = round(res[0],2)
    new_line['KS-pvalue'] = round(res[1],2)
    new_line['KL-entropy'] = round(Sp,2)
    new_line['KL-div'] = round(kl_res,2)

    summary_stat = summary_stat.append(new_line,ignore_index = True)

  ### Add GNU 000 for referencfe
  x = np.array(ens_df[my_var])
  y = np.array(add_test_df[my_var])
  bins = np.linspace(np.nanmin(np.hstack((x,y))),np.nanmax(np.hstack((x,y))),40)
  
  #p = plt.hist(x,bins=bins)[0]
  #q = plt.hist(y,bins=bins)[0]
  p = np.histogram(x,bins = bins)[0]
  q = np.histogram(y,bins = bins)[0]
  p = p/p.sum(axis=0, keepdims=True) 
  q = q/q.sum(axis=0, keepdims=True) 
  #z = np.array(test_df[my_var])



  # Note slight difference in the final result compared to Dawny33
  #print('This is GNU.000')
  res = stats.ks_2samp(x,y)
  #print('KS-statistics: ',res)
  #kl_res = scipy.special.kl_div(x,y)
  kl_res = KL(p,q)
  Sp = stats.entropy(np.array(x))
  #print('KL-entropy of A: ',Sp)
  #print('KL-divergence: ',kl_res)

  new_line = one_line.copy()
  new_line['Dist_A'] = str(add_test_df.Label.iloc[0])+'.'+str(add_test_df.Simulation.iloc[0])
  new_line['Dist_B'] = str(len(run_list))+' '+str(ens_df.Label.iloc[0])
  new_line['KS-stats'] = round(res[0],2)
  new_line['KS-pvalue'] = round(res[1],2)
  new_line['KL-entropy'] = round(Sp,2)
  new_line['KL-div'] = round(kl_res,2)
  

  summary_stat = summary_stat.append(new_line,ignore_index = True)
  print('Reject Null Hypothesis ('+str(alpha*100)+'% Confidence level) if D > ',round(np.sqrt(-np.log(alpha/2)/2)*np.sqrt(2600/2500/100),3))
  return summary_stat[['Dist_A','Dist_B','KS-stats','KS-pvalue','KL-div']].transpose()

stats_table(my_var,ens_df,add_test_df)

In [ ]:
var_list = column_list
my_label = 'Intel'
#add_test_df = df[(df.Label == my_label) & (df.Simulation.isin(['001']))  & (df.nyear >= nyear - use_year)]
add_test_df = df[(df.Label == 'GNU') & (df.Simulation.isin(['001']))  & (df.nyear >= nyear - use_year)]
ens_df = df[(df.Label == my_label)  & (df.nyear >= nyear - use_year)]



@interact(my_var=var_list)
def make_table_for(my_var='TS'):
    my_stats_table = stats_table(my_var,ens_df,add_test_df)
    display(my_stats_table)

distribution of KL and KS.


In [ ]:
ens_df.SNOWHICE

#### KL Divergence

Test against Gaussian. 

In [ ]:

ens_df = gnu_df
add_test_df =  df[(df.Label == 'GNU') & (df.Simulation == '004')]
my_var='TS'

def stats_table(my_var,ens_df,add_test_df):
  # my_var is for the variable to show
  # add_test_df is the one member to add to comparison (in addition to all Test Simulations)
  # ens_df is the ensemble to test against (Dist B)
  res = stats.ks_2samp(gnu_df[my_var],intel_df[my_var])
  ### Test all test runs
  summary_stat = pd.DataFrame()
  one_line = {
      'Dist_A':'',
      'Dist_B':'',
      'KS-stats':0,
      'KS-pvalue':0,
      'KL-div':0,
      'KL-entropy':0
  }
  test_list
  run_list = np.unique(ens_df.Simulation)

  for my_test in test_list:
    new_line = one_line.copy()
    test_df = df[df.Simulation == my_test]
    # Change to Intel to test against Intel Ensemble
    x = np.array(ens_df[my_var])
    y = np.array(test_df[my_var])
    bins = np.linspace(np.nanmin(np.hstack((x,y))),np.nanmax(np.hstack((x,y))),40)


    p = np.histogram(x,bins = bins)[0]
    q = np.histogram(y,bins = bins)[0]
    #p = plt.hist(x,bins=bins)[0]
    #q = plt.hist(y,bins=bins)[0]
    p = p/p.sum(axis=0, keepdims=True) 
    q = q/q.sum(axis=0, keepdims=True) 

    #print('This is test run: ',my_test)
    res = stats.ks_2samp(x,y)
    #print('KS-statistics: ',res)
    Sp = stats.entropy(np.array(x))
    #print('KL-entropy of A: ',Sp)
    kl_res = KL(p,q)
    #print('KL-divergence: ',kl_res)
    new_line['Dist_A'] = my_test
    new_line['Dist_B'] = str(len(run_list))+' '+str(ens_df.Label.iloc[0])
    new_line['KS-stats'] = round(res[0],2)
    new_line['KS-pvalue'] = round(res[1],2)
    new_line['KL-entropy'] = round(Sp,2)
    new_line['KL-div'] = round(kl_res,2)

    summary_stat = summary_stat.append(new_line,ignore_index = True)

  ### Add GNU 000 for referencfe
  x = np.array(ens_df[my_var])
  y = np.array(add_test_df[my_var])
  bins = np.linspace(np.nanmin(np.hstack((x,y))),np.nanmax(np.hstack((x,y))),40)
  
  #p = plt.hist(x,bins=bins)[0]
  #q = plt.hist(y,bins=bins)[0]
  p = np.histogram(x,bins = bins)[0]
  q = np.histogram(y,bins = bins)[0]
  p = p/p.sum(axis=0, keepdims=True) 
  q = q/q.sum(axis=0, keepdims=True) 
  #z = np.array(test_df[my_var])



  # Note slight difference in the final result compared to Dawny33
  #print('This is GNU.000')
  res = stats.ks_2samp(x,y)
  #print('KS-statistics: ',res)
  #kl_res = scipy.special.kl_div(x,y)
  kl_res = KL(p,q)
  Sp = stats.entropy(np.array(x))
  #print('KL-entropy of A: ',Sp)
  #print('KL-divergence: ',kl_res)

  new_line = one_line.copy()
  new_line['Dist_A'] = str(ens_df.Label.iloc[0])+'.'+str(add_test_df.Simulation.iloc[0])
  new_line['Dist_B'] = str(len(run_list))+' '+str(ens_df.Label.iloc[0])
  new_line['KS-stats'] = round(res[0],2)
  new_line['KS-pvalue'] = round(res[1],2)
  new_line['KL-entropy'] = round(Sp,2)
  new_line['KL-div'] = round(kl_res,2)
  

  summary_stat = summary_stat.append(new_line,ignore_index = True)
  return summary_stat[['Dist_A','Dist_B','KS-stats','KS-pvalue','KL-div']].transpose()

stats_table(my_var,ens_df,add_test_df)

https://stats.stackexchange.com/questions/111445/analysis-of-kullback-leibler-divergence

https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence

In [ ]:
x = np.array(intel_df[my_var])
y = np.array(gnu_df[my_var])
# Normalize to pdf
bins = np.linspace(np.nanmin(np.hstack((x,y))),np.nanmax(np.hstack((x,y))),40)
 
p = plt.hist(x,bins=bins)[0]
q = plt.hist(y,bins=bins)[0]
p = p/p.sum(axis=0, keepdims=True) 
q = q/q.sum(axis=0, keepdims=True) 
#z = np.array(test_df[my_var])

S = stats.entropy(p,q)
#kl_res = scipy.special.kl_div(p,q)
kl_res = KL(p,q)
Sp = stats.entropy(p)
Sq = stats.entropy(q)

print(Sp,Sq,kl_res)

fig = plt.figure(3)
#plt.hist(x, bins=bins, density=True, histtype='step', cumulative=False,
#        label='Intel',lw=2)
#plt.hist(y, bins=bins, density=True, histtype='step', cumulative=False,
#        label='GNU',lw=2)
#plt.hist(kl_res,bins=bins, density=True, histtype='step', cumulative=False,
#        label='KL Div',lw=2)
sns.distplot(x, hist = False, kde = True,
                 kde_kws = {'linewidth': 2},
                 label = 'Intel')

sns.distplot(y, hist = False, kde = True,
                 kde_kws = {'linewidth': 2},
                 label = 'GNU')

plt.legend()
#plt.title('KL(P||Q) = %1.2f' % S)


#### DataFrame for KS-stats and KL-div


In [ ]:
ens_df = intel_df
add_test_df =  df[(df.Label == 'GNU')]
my_var='TS'

def stats_table(my_var,ens_df,add_test_df):
  # my_var is for the variable to show
  # add_test_df is the one member to add to comparison (in addition to all Test Simulations)
  # ens_df is the ensemble to test against (Dist B)
  res = stats.ks_2samp(gnu_df[my_var],intel_df[my_var])
  ### Test all test runs
  summary_stat = pd.DataFrame()
  one_line = {
      'Dist_A':'',
      'Dist_B':'',
      'KS-stats':0,
      'KS-pvalue':0,
      'KL-div':0,
      'KL-entropy':0
  }
  test_list
  run_list = np.unique(ens_df.Simulation)

  for my_test in test_list:
    new_line = one_line.copy()
    test_df = df[df.Simulation == my_test]
    # Change to Intel to test against Intel Ensemble
    x = np.array(ens_df[my_var])
    y = np.array(test_df[my_var])
    bins = np.linspace(np.nanmin(np.hstack((x,y))),np.nanmax(np.hstack((x,y))),40)


    p = np.histogram(x,bins = bins)[0]
    q = np.histogram(y,bins = bins)[0]
    #p = plt.hist(x,bins=bins)[0]
    #q = plt.hist(y,bins=bins)[0]
    p = p/p.sum(axis=0, keepdims=True) 
    q = q/q.sum(axis=0, keepdims=True) 

    #print('This is test run: ',my_test)
    res = stats.ks_2samp(x,y)
    #print('KS-statistics: ',res)
    Sp = stats.entropy(np.array(x))
    #print('KL-entropy of A: ',Sp)
    kl_res = KL(p,q)
    #print('KL-divergence: ',kl_res)
    new_line['Dist_A'] = my_test
    new_line['Dist_B'] = str(len(run_list))+' '+str(ens_df.Label.iloc[0])
    new_line['KS-stats'] = round(res[0],2)
    new_line['KS-pvalue'] = round(res[1],2)
    new_line['KL-entropy'] = round(Sp,2)
    new_line['KL-div'] = round(kl_res,2)

    summary_stat = summary_stat.append(new_line,ignore_index = True)

  #simu_list = np.unique(ens_df.Simulation)
  simu_list = ['001']
  ### Add GNU 000 for referencfe
  for my_simu in simu_list:
    add_test_df = add_test_df#ens_df[ens_df.Simulation == my_simu]
    x = np.array(ens_df[my_var])
    y = np.array(add_test_df[my_var])
    bins = np.linspace(np.nanmin(np.hstack((x,y))),np.nanmax(np.hstack((x,y))),40)
    
    #p = plt.hist(x,bins=bins)[0]
    #q = plt.hist(y,bins=bins)[0]
    p = np.histogram(x,bins = bins)[0]
    q = np.histogram(y,bins = bins)[0]
    p = p/p.sum(axis=0, keepdims=True) 
    q = q/q.sum(axis=0, keepdims=True) 
    #z = np.array(test_df[my_var])



    # Note slight difference in the final result compared to Dawny33
    #print('This is GNU.000')
    res = stats.ks_2samp(x,y)
    #print('KS-statistics: ',res)
    #kl_res = scipy.special.kl_div(x,y)
    kl_res = KL(p,q)
    Sp = stats.entropy(np.array(x))
    #print('KL-entropy of A: ',Sp)
    #print('KL-divergence: ',kl_res)

    new_line = one_line.copy()
    new_line['Dist_A'] = str(ens_df.Label.iloc[0])+'.'+str(add_test_df.Simulation.iloc[0])
    new_line['Dist_B'] = str(len(run_list))+' '+str(ens_df.Label.iloc[0])
    new_line['KS-stats'] = round(res[0],2)
    new_line['KS-pvalue'] = round(res[1],2)
    new_line['KL-entropy'] = round(Sp,2)
    new_line['KL-div'] = round(kl_res,2)
    

    summary_stat = summary_stat.append(new_line,ignore_index = True)
  return summary_stat#[['Dist_A','Dist_B','KS-stats','KS-pvalue','KL-div']].transpose()

summary_stat[['Dist_A','Dist_B','KS-stats','KS-pvalue','KL-div']].transpose()
my_stat = stats_table(my_var,ens_df,add_test_df)
ens_stat = my_stat[my_stat.Dist_A.str.contains('Intel')]


In [ ]:
ens_stat

In [ ]:
### Plot the distribution of three stats in three subplots
fig = plt.figure(figsize = (8,8))
for idx,my_stat in enumerate(['KS-stats','KS-pvalue','KL-div']):
  ax = plt.subplot(2,2,idx+1)
  #ax.set_title(my_stat)
  x = ens_stat[my_stat]
  bins = np.linspace(np.nanmin(x),np.nanmax(x),10)
  sns.distplot(x,bins= bins)


In [ ]:
newdf = pd.DataFrame()
ens_df = intel_df
add_test_df =  df[(df.Label == 'GNU')]
for my_var in column_list:
  stat_df = stats_table(my_var,ens_df,add_test_df)
  stat_df['Variable'] = my_var
  newdf = newdf.append(stat_df)

In [ ]:

for itest, my_test in enumerate(test_list):
  subset = newdf[newdf.Dist_A.str.contains(my_test)]
  print('% rejections of ',my_test)
  print(len(subset[subset['KS-pvalue'] < ci])*1./len(subset))


In [ ]:
my_test = test_list[1]
for my_var in column_list:
  subset = newdf[~(newdf.Dist_A.str.contains('Intel')) & (newdf['Variable'] == my_var)]
  if len(subset[subset['KS-pvalue'] < ci]) == 0:
    print(my_var)
    #print(subset[subset['KS-pvalue'] < ci].Dist_A)
    #print(len(subset[subset['KS-pvalue'] < ci]))



#### Plot heatmap

In [ ]:
newdf.replace(to_replace='Intel.000',value = 'GNU',inplace =True)
newdf


In [ ]:
### Draw heatmap:
remove_added = False

data_df = newdf.pivot_table(values = 'KS-pvalue',index = 'Variable',columns='Dist_A')
if remove_added:
  data_df = data_df.drop(columns = 'GNU')
  new_test_list = ['rh-min-low', 'albice00', 'cpl-bug','0nu', 'fma','expand',  'no-opt', 'rand-mt']
else:
  new_test_list = ['rh-min-low', 'albice00', 'cpl-bug','0nu', 'fma','expand',  'no-opt', 'rand-mt','GNU']

var_df_removed = var_df[var_df.variable.isin(removed_column_list)].drop(columns=['Unnamed: 0','index'])
var_df_removed['Variable'] = var_df_removed['variable']
data_df = pd.merge(data_df,var_df_removed,on = 'Variable')
data_df

In [ ]:

my_type_list = np.unique(data_df['my_type'])
mapping = {'Aerosol':1,'Precipitation':2,'Cloud':3,'Temperature':4,'Pressure':5,'Flux':6,'Fraction':7,'Height':8,'Transport':9}
data_df['my_color'] = data_df['my_type'].map(mapping)
#newdf['my_color'] = newdf['my_type'].map(lambda x, y: y, my_type_list, range(9))
my_type_list = [my_type for my_type in mapping]
my_type_list

In [ ]:
#data = np.random.randn(6, 6)
nvar = len(removed_column_list)
nmodes = len(new_test_list)

data = np.zeros((nvar,nmodes))
ks_array = np.zeros((nvar,nmodes))

for itest,my_test in enumerate(new_test_list):
  #data_df
  df_reorder = data_df.sort_values(by = my_test,ascending=True)
  data[:,itest] = df_reorder['my_color']
  ks_array[:,itest] = df_reorder[my_test]
  #my_sig = 'pc_sig'+str(imode+1)



In [ ]:
#data[:,0]
ks_array[:,0]

In [ ]:
from matplotlib.colors import from_levels_and_colors
from matplotlib.colorbar import ColorbarBase
from matplotlib import colors
import matplotlib as mpl
from skimage.segmentation import mark_boundaries
if remove_added:
  fig, ax = plt.subplots(figsize=(10,10))
  #cmap= colors.ListedColormap(['#fee0b6','#2166ac','#67a9cf','#b2182b','#984ea3','#fc8d59','#999999','#7fbf7b','#e0e0e0'])

  cmap= colors.ListedColormap(['#fee0b6','#2166ac','#67a9cf','#b2182b','#984ea3','#fc8d59','#999999','#bababa','#31a354'])
  bounds = np.arange(0.,10.,1)
  norm = colors.BoundaryNorm(bounds, cmap.N, clip=True)


  #plt.cm.PiYG
  '''
  qrates = np.array(my_type_list)
  norm = mpl.colors.BoundaryNorm(np.linspace(1, 10, 10), 9)
  fmt = mpl.ticker.FuncFormatter(lambda x, pos: qrates[:][norm(x)])

  x = np.arange(0.5,10.5,10)
  im = ax.imshow(data, cmap=cmap,  norm = norm, extent=[0.5,9.5,len(removed_column_list)+0.5,-0.5])
  ax.contourf(alpha_array, 1, hatches=['//', ''], alpha=0,extent=[0.5,9.5,-0.5,len(removed_column_list)+0.5])
  '''
  new_alpha = ks_array.copy()
  new_alpha = np.vstack((new_alpha[0,:],new_alpha,new_alpha[-1,:]))
  new_alpha = np.hstack((new_alpha[:,0].reshape(len(new_alpha),1),new_alpha,new_alpha[:,-1].reshape(len(new_alpha),1)))
  my_shape = np.shape(new_alpha)

  qrates = np.array(my_type_list)
  norm = mpl.colors.BoundaryNorm(np.linspace(1, 10, 10), 9)
  fmt = mpl.ticker.FuncFormatter(lambda x, pos: qrates[:][norm(x)])



  im = ax.imshow(data, cmap=cmap,  norm = norm)#, extent=[0.5,9.5,len(removed_column_list)+0.5,-0.5])
  ax.contourf(new_alpha < 0.05, 1, hatches=['//', ''], alpha=0,origin = 'image',extent=[-1.5,8.5,len(removed_column_list)+1.5,-1.5])
  ax.set_xlim(-0.5,7.5)
  ax.set_ylim(len(removed_column_list)-0.5,-0.5)

  ax.set_xticks(range(8))
  #ax.set_xticklabels(labels=[str(imode+1) for imode in range(8)])
  ax.set_xticklabels(labels=new_test_list,rotation=45)

  ax.set_xlabel('Test runs') ## Zero means largest PC mode

  ax.set_ylabel('Ascending order of KS test p-values') ## Zero means largest PC mode
  ax.set_aspect(0.1)


  cbar_kw=dict(ticks=np.arange(0.5, 10.5,1), format=fmt,norm=norm)
  cbar = ax.figure.colorbar(im,  **cbar_kw)
  cbar.ax.set_ylabel("Type", rotation=-90, va="bottom",fontsize = 12)

  plt.savefig(local_dir+'KS_test_runs_physics.png',dpi = 500)
else:
  print('KS heatmap not shown')

HEatmap, sorted by type, colored by KS pvalue. 

In [ ]:
data_df['sort_order'] = 8
data_df['sort_order'].loc[data_df.my_type == 'Flux'] = 0
data_df['sort_order'].loc[data_df.my_type == 'Cloud'] = 1
data_df['sort_order'].loc[data_df.my_type == 'Aerosol'] = 2
data_df['sort_order'].loc[data_df.my_type == 'Precipitation'] = 3
data_df['sort_order'].loc[data_df.my_type == 'Height'] = 4
data_df['sort_order'].loc[data_df.my_type == 'Temperature'] = 5
data_df['sort_order'].loc[data_df.my_type == 'Pressure'] = 6
data_df['sort_order'].loc[data_df.my_type == 'Transport'] = 7

data_df = data_df.sort_values(by=['sort_order','my_type'])
data_df = data_df.reset_index()

data_df

In [ ]:
remove_var = True

var_df = pd.read_csv(local_dir+'CESM_variable_type.csv')
var_df['my_type'] = var_df['type']
new_var_df = var_df.copy()
new_var_df['sort_order'] = 8
new_var_df['sort_order'].loc[new_var_df.my_type == 'Flux'] = 0
new_var_df['sort_order'].loc[new_var_df.my_type == 'Cloud'] = 1
new_var_df['sort_order'].loc[new_var_df.my_type == 'Aerosol'] = 2
new_var_df['sort_order'].loc[new_var_df.my_type == 'Precipitation'] = 3
new_var_df['sort_order'].loc[new_var_df.my_type == 'Height'] = 4
new_var_df['sort_order'].loc[new_var_df.my_type == 'Temperature'] = 5
new_var_df['sort_order'].loc[new_var_df.my_type == 'Pressure'] = 6
new_var_df['sort_order'].loc[new_var_df.my_type == 'Transport'] = 7

nine_type_list = ['Flux','Cloud','Aerosol','Precipitation','Height','Temperature','Pressure','Transport','Fraction']

new_var_df = new_var_df.sort_values(by=['sort_order','my_type'])
new_var_df = new_var_df.reset_index()
sorted_column_list = new_var_df.variable
sorted_type_list = new_var_df.my_type

sorted_column_list = list(sorted_column_list)
# Remove three variables with bump start: 'OCNFRAC','ICEFRAC','SNOWHICE'
# Remove three constant variables: 'LANDFRAC','PHIS','SOLIN'
#remove_var_list =  ['OCNFRAC','ICEFRAC','SNOWHICE','LANDFRAC','PHIS','SOLIN']
#remove_var_list =  ['EMISCLD','OCNFRAC','ICEFRAC','SNOWHICE','LANDFRAC','PHIS','SOLIN']
for my_col in remove_var_list:
  sorted_column_list.remove(my_col)
np.array(sorted_column_list,dtype=str)

if remove_var:
  column_list = sorted_column_list

In [ ]:
#data = np.random.randn(6, 6)
nvar = len(removed_column_list)
nmodes = len(new_test_list)

data = np.zeros((nvar,nmodes))
ks_array = np.zeros((nvar,nmodes))

for itest,my_test in enumerate(new_test_list):
  #data_df
  df_reorder = data_df.copy()#.sort_values(by = my_test,ascending=True)
  data[:,itest] = df_reorder['my_color']
  ks_array[:,itest] = df_reorder[my_test]
  #my_sig = 'pc_sig'+str(imode+1)


In [ ]:
### Color coding by KS p-value

fig, ax = plt.subplots(figsize=(10,10))
#YlBu = np.array(['#ffffd9','#edf8b1','#c7e9b4','#7fcdbb','#41b6c4','#1d91c0','#225ea8','#0c2c84'])[::-1]
#cmap= colors.ListedColormap(YlBu)
#bounds = [0., 0.001, 0.005, 0.01,0.05,0.1,0.5,0.75,1.]

YlBu = np.array(['#f7f7f7','#ffffd9','#edf8b1','#c7e9b4','#7fcdbb','#41b6c4','#1d91c0','#225ea8','#0c2c84'])[::-1]
cmap= colors.ListedColormap(YlBu)
bounds = [0., 0.001, 0.005, 0.01,0.05,0.1,0.2,0.5,0.9,1.]
norm = colors.BoundaryNorm(bounds, cmap.N, clip=True)


#plt.cm.PiYG
'''
qrates = np.array(my_type_list)
norm = mpl.colors.BoundaryNorm(np.linspace(1, 10, 10), 9)
fmt = mpl.ticker.FuncFormatter(lambda x, pos: qrates[:][norm(x)])

x = np.arange(0.5,10.5,10)
im = ax.imshow(data, cmap=cmap,  norm = norm, extent=[0.5,9.5,len(removed_column_list)+0.5,-0.5])
ax.contourf(alpha_array, 1, hatches=['//', ''], alpha=0,extent=[0.5,9.5,-0.5,len(removed_column_list)+0.5])
'''
new_alpha = ks_array.copy()
new_alpha = np.vstack((new_alpha[0,:],new_alpha,new_alpha[-1,:]))
new_alpha = np.hstack((new_alpha[:,0].reshape(len(new_alpha),1),new_alpha,new_alpha[:,-1].reshape(len(new_alpha),1)))
my_shape = np.shape(new_alpha)

qrates = [str(my_cbar_tick) for my_cbar_tick in bounds]
norm = mpl.colors.BoundaryNorm(bounds,9)
fmt = mpl.ticker.FuncFormatter(lambda x, pos: qrates[:][norm(x)])


#im = ax.imshow(ks_array, cmap= 'RdBu_r', vmin = 0., vmax = 1)#, extent=[0.5,9.5,len(removed_column_list)+0.5,-0.5])
im = ax.imshow(ks_array, cmap= cmap, norm=norm)#, extent=[0.5,9.5,len(removed_column_list)+0.5,-0.5])
#ax.contourf(new_alpha < 0.05, 1, hatches=['//', ''], alpha=0,origin = 'image',extent=[-1.5,8.5,len(removed_column_list)+1.5,-1.5])
ax.set_xlim(-0.5,len(new_test_list)-0.5)
ax.set_ylim(len(removed_column_list)-0.5,-0.5)

ax.set_xticks(range(len(new_test_list)))
#ax.set_xticklabels(labels=[str(imode+1) for imode in range(8)])
ax.set_xticklabels(labels=new_test_list,rotation=45,fontsize = fs-1)
ytick_locs = ([np.mean(np.array(data_df[data_df.my_type == my_type].index)) for my_type in (nine_type_list)])
ytick_sep_locs = ([data_df[data_df.my_type == my_type].index[0]-0.5 for my_type in nine_type_list])
ytick_sep_locs = list(ytick_sep_locs)
ytick_sep_locs.append(ytick_sep_locs[-1]+1)
ytick_labels = ([str(my_ytick) for my_ytick in nine_type_list ])

ax.vlines([0.5,1.5,2.5,3.5,4.5,5.5,6.5],-0.5,127.5,linewidth=1)
ax.vlines(3.5,-0.5,127.5,linewidth=2)
if not remove_added:
  ax.vlines(7.5,-0.5,127.5,linewidth=2)
ax.hlines(ytick_sep_locs,-0.5,len(new_test_list)-0.5,linewidth=1)

ax.set_xlabel('All runs',fontsize = fs) ## Zero means largest PC mode

ax.set_ylabel('Variables ordered by type',fontsize = fs-2) ## Zero means largest PC mode
ax.set_yticks(ytick_locs)
ax.set_yticklabels(ytick_labels,fontsize = fs-1)
ax.set_yticks(ytick_sep_locs,minor = True)
ax.tick_params(axis='y', which='major',length=0)
ax.tick_params(axis='y', which='minor',length=5)

ax.set_aspect(0.1)


#cbar_kw=dict(ticks=np.arange(0,1), format=fmt)
cbar = ax.figure.colorbar(im)
cbar.ax.set_ylabel("K-S p-value", rotation=-90, va="bottom",fontsize = fs-2)

plt.savefig(local_dir+'KS_test_runs_physics.png',dpi = 500)

In [ ]:
### Version ordered by PC. 
sorted_column_list_pc = np.load(local_dir+'sorted_column_list_pc.npy')

sorted_column_df = pd.DataFrame(sorted_column_list_pc,columns = ['variable'])
new_data_df = sorted_column_df.merge(data_df,how = 'left')
new_data_df

In [ ]:
data_df[data_df.variable == 'FSDSC']

In [ ]:
#data = np.random.randn(6, 6)
nvar = len(removed_column_list)
nmodes = len(new_test_list)

data = np.zeros((nvar,nmodes))
ks_array = np.zeros((nvar,nmodes))

for itest,my_test in enumerate(new_test_list):
  #data_df
  df_reorder = new_data_df.copy()#.sort_values(by = my_test,ascending=True)
  data[:,itest] = df_reorder['my_color']
  ks_array[:,itest] = df_reorder[my_test]
  #my_sig = 'pc_sig'+str(imode+1)


In [ ]:
my_test1 = new_test_list[5]
my_test2 = new_test_list[2]
print(my_test1,my_test2)
np.array(data_df[(data_df[my_test1] < 0.001) & (data_df[my_test2] > 0.001)].variable)

In [ ]:
### KS p-value ordered by PC

fig, ax = plt.subplots(figsize=(10,10))

YlBu = np.array(['#f7f7f7','#ffffd9','#edf8b1','#c7e9b4','#7fcdbb','#41b6c4','#1d91c0','#225ea8','#0c2c84'])[::-1]
cmap= colors.ListedColormap(YlBu)
bounds = [0., 0.001, 0.005, 0.01,0.05,0.1,0.2,0.5,0.9,1.]
norm = colors.BoundaryNorm(bounds, cmap.N, clip=True)


#plt.cm.PiYG
'''
qrates = np.array(my_type_list)
norm = mpl.colors.BoundaryNorm(np.linspace(1, 10, 10), 9)
fmt = mpl.ticker.FuncFormatter(lambda x, pos: qrates[:][norm(x)])

x = np.arange(0.5,10.5,10)
im = ax.imshow(data, cmap=cmap,  norm = norm, extent=[0.5,9.5,len(removed_column_list)+0.5,-0.5])
ax.contourf(alpha_array, 1, hatches=['//', ''], alpha=0,extent=[0.5,9.5,-0.5,len(removed_column_list)+0.5])
'''
new_alpha = ks_array.copy()
new_alpha = np.vstack((new_alpha[0,:],new_alpha,new_alpha[-1,:]))
new_alpha = np.hstack((new_alpha[:,0].reshape(len(new_alpha),1),new_alpha,new_alpha[:,-1].reshape(len(new_alpha),1)))
my_shape = np.shape(new_alpha)

qrates = [str(my_cbar_tick) for my_cbar_tick in bounds]
norm = mpl.colors.BoundaryNorm(bounds,len(YlBu))
fmt = mpl.ticker.FuncFormatter(lambda x, pos: qrates[:][norm(x)])


#im = ax.imshow(ks_array, cmap= 'RdBu_r', vmin = 0., vmax = 1)#, extent=[0.5,9.5,len(removed_column_list)+0.5,-0.5])
im = ax.imshow(ks_array, cmap= cmap, norm=norm)#, extent=[0.5,9.5,len(removed_column_list)+0.5,-0.5])
#ax.contourf(new_alpha < 0.05, 1, hatches=['//', ''], alpha=0,origin = 'image',extent=[-1.5,8.5,len(removed_column_list)+1.5,-1.5])
ax.set_xlim(-0.5,len(new_test_list)-0.5)
ax.set_ylim(len(removed_column_list)-0.5,-0.5)

ax.set_xticks(range(len(new_test_list)))
#ax.set_xticklabels(labels=[str(imode+1) for imode in range(8)])
ax.set_xticklabels(labels=new_test_list,rotation=45)
ytick_locs = list(np.arange(7.5,120,15) - 0.5)
ytick_locs.append(123)

ytick_sep_locs = np.arange(0,128,15) - 0.5

ytick_labels = (['mode'+str(my_mode+1) for my_mode in range(9) ])

ax.vlines([0.5,1.5,2.5,3.5,4.5,5.5,6.5],-0.5,127.5,linewidth=1)
ax.vlines(3.5,-0.5,127.5,linewidth=2)
if not remove_added:
  ax.vlines(7.5,-0.5,127.5,linewidth=2)
ax.hlines(ytick_sep_locs,-0.5,len(new_test_list)-0.5,linewidth=1)

ax.set_xlabel('Test runs') ## Zero means largest PC mode

ax.set_ylabel('Variables ordered by PC modes') ## Zero means largest PC mode
ax.set_yticks(ytick_locs)
ax.set_yticklabels(ytick_labels)
ax.set_yticks(ytick_sep_locs,minor = True)
ax.tick_params(axis='y', which='major',length=0)
ax.tick_params(axis='y', which='minor',length=5)

ax.set_aspect(0.1)


#cbar_kw=dict(ticks=np.arange(0,1), format=fmt)
cbar = ax.figure.colorbar(im)
cbar.ax.set_ylabel("K-S p-value", rotation=-90, va="bottom",fontsize = 12)

plt.savefig(local_dir+'KS_test_runs_physics_pc.png',dpi = 500)

In [ ]:
np.unique([np.mean(np.array(data_df[data_df.my_type == my_type].index)) for my_type in (sorted_type_list)])


## Testing same mean and variance

#### T-test for same mean

In [ ]:

for my_var in column_list:
  X = intel_df[intel_df.Simulation != '000'][my_var]
  Y = gnu_df[gnu_df.Simulation != '000'][my_var]
  #Y = df[df.Simulation == my_test][my_var]
  df1 = len(X) - 1
  df2 = len(Y) - 1

  alpha = 0.05 #Or whatever you want your alpha to be.
  t_stat, p_value = scipy.stats.ttest_ind(X,Y)
  if p_value < alpha:
    print(my_var)
    #print('Reject the hypothesis',my_var,p_value)

#### F-test for same variance

In [ ]:
test_df.Simulation

In [ ]:
my_test = test_list[7]
print('Test run: ',my_test)
for my_var in column_list:
  X = intel_df[intel_df.Simulation != '000'][my_var]
  #Y = gnu_df[gnu_df.Simulation != '000'][my_var]
  Y = df[df.Simulation == my_test][my_var]
  F = np.var(X) / np.var(Y)
  df1 = len(X) - 1
  df2 = len(Y) - 1

  alpha = 0.05 #Or whatever you want your alpha to be.
  p_value = scipy.stats.f.cdf(F, df1, df2)
  if p_value < alpha:
    print(my_var)
    #print('Reject the hypothesis',my_var,p_value)

#### Year start to diverge?

Answering the question by Dorit: In which year does the difference start to show up? 

In [ ]:
# Always remove Intel 006
matched = True
if not matched:
  gnu_df = df[(df.Label == 'GNU')]
  intel_df = df[(df.Label == 'Intel')]
  #intel_df = df[(df.Label == 'Intel') & (df.Simulation != '006')]
  intel_perc = np.nanpercentile(df[(df.Label == 'Intel')][my_var],my_perc)
else:
  match_list = [str(simu).zfill(3) for simu in range(25)]
  #match_list.remove('006')
  gnu_df = df[(df.Label == 'GNU') & (df.Simulation.isin(match_list))]
  intel_df = df[(df.Label == 'Intel') & (df.Simulation.isin(match_list))]

In [ ]:
sns.lineplot(x = "nyear", y = "TS", data = intel_df,label = 'Intel')
sns.lineplot(x = "nyear", y = "TS", data = gnu_df,label='GNU')
plt.legend()

In [ ]:
sns.lineplot(x = "nyear", y = "TS", data = intel_df,label = 'Intel')
sns.lineplot(x = "nyear", y = "TS", data = df[df.Simulation == test_list[-1]],label=test_list[-1])
plt.legend(frameon=False)

In [ ]:
my_var = 'TS'
ts = np.array(df[df.Simulation == test_list[-1]][my_var]) - intel_df.groupby('nyear')[my_var].mean()

ts[abs(ts) > 0.05 ]

#### KS-test for each year GNU runs

In [ ]:
intel_df = df[(df.Label == 'Intel') & (df.Simulation != '000') ]
gnu_df = df[(df.Label == 'GNU') & (df.Simulation != '000')]
intel_df.Simulation


In [ ]:
### How many variables fails the 5% KS-test in each year (or plot p-value across time)
nyear = 100
confidence_level = 0.05
fail_ts = np.zeros(nyear)
pvalue_ts = np.zeros(nyear)

for yr in range(nyear):
  num_fail = 0
  pvalue_list = []
  for my_var in column_list:
    x = intel_df[(intel_df.nyear == yr)][my_var]
    y = gnu_df[gnu_df.nyear == yr][my_var]
    #y = df[(df.Simulation.str.contains(test_list[0]))][my_var]
    ks_res = stats.ks_2samp(x,y)
    pvalue = round(ks_res[1],2)
    pvalue_list.append(pvalue)
    if pvalue < confidence_level:
      num_fail = num_fail+1
      if yr <= 9:
        print('This is year ',yr,my_var)
  fail_ts[yr] = num_fail/134.
  pvalue_ts[yr] = np.nanmean(pvalue_list)



In [ ]:
fig = plt.figure(1,figsize = (8,10))
ax= plt.subplot(2,1,1)
sns.lineplot(range(nyear),fail_ts*100,label = 'GNU')
plt.xlabel("Years")
plt.ylabel("Percent")
plt.legend(frameon=False)
plt.title("Percent of variables failing  {:.2f} KS-test".format(confidence_level))


ax= plt.subplot(2,1,2)
pvalue_std = np.nanstd(pvalue_ts)
sns.lineplot(range(nyear),pvalue_ts,label = 'GNU')
plt.xlabel("Years")
plt.ylabel("p-value")
plt.legend(frameon=False)
plt.title("Mean p-value of KS-test")
plt.hlines(np.mean(pvalue_ts) + 1*pvalue_std,0,100,linestyle = '--')
plt.hlines(np.mean(pvalue_ts) - 1*pvalue_std,0,100,linestyle = '--')


In [ ]:
for my_var in column_list:
  num_fail = 0
  for yr in range(nyear):
    x = intel_df[(intel_df.nyear == yr)][my_var]
    y = gnu_df[gnu_df.nyear == yr][my_var]
    #y = df[(df.Simulation.str.contains(test_list[0]))][my_var]
    ks_res = stats.ks_2samp(x,y)
    pvalue = round(ks_res[1],2)
    if pvalue < confidence_level:
      num_fail = num_fail+1
      #if yr <= 9:
      #  print('This is year ',yr,my_var)
  fail_ts[yr] = num_fail/100.

In [ ]:
fail_ts


#### Bootstrapping of time series

In [ ]:
my_run = df[(df.Simulation == 'rh-min-low')]
plt.plot(range(100),my_run['VT'],label='rh-min-low')

my_intel = df[(df.Simulation == '001') & (df.Label == 'Intel')]
plt.plot(range(100),my_intel['VT'],label='Intel.001')

plt.legend()

In [ ]:
### Shuffle Intel

#plt.plot(range(100),my_run['VT'],label='rh-min-low')

#my_intel = df[(df.Simulation == '001') & (df.Label == 'Intel')]
#plt.plot(range(100),my_intel['VT'],label='Intel.001')


fig = plt.figure(1,figsize = (12,6))
mean_intel = []
std_intel = []
nyear = 100
use_year = 100
my_var = 'VT'
intel_df = df[df.Label == 'Intel']
nshuffle = 10
div_year = int(use_year/2)

my_test = 'rh-min-low'
my_run = df[(df.Simulation == my_test) & (df.nyear >= nyear - use_year)]

for my_simu in np.unique(intel_df.Simulation):
  intel_ts = np.array(intel_df[(intel_df.Simulation == my_simu) & (intel_df.nyear >= nyear - use_year)][my_var])
  for ishuffle in range(nshuffle):
    np.random.shuffle(intel_ts)
    new_ts = intel_ts.copy()
    
    mean_intel.append(np.nanmean(new_ts[:div_year])/np.nanmean(new_ts[div_year:]))
    std_intel.append(np.nanstd(new_ts[:div_year])/np.nanstd(new_ts[div_year:]))

### Shuffle rh-min-low
mean_test = []
std_test = []
rh_ts = np.array(my_run['VT'])
print(rh_ts)
for ishuffle in range(1000):
  np.random.shuffle(rh_ts)
  new_ts = rh_ts.copy()
  mean_test.append(np.nanmean(new_ts[:div_year])/np.nanmean(new_ts[div_year:]))
  std_test.append(np.nanstd(new_ts[:div_year])/np.nanstd(new_ts[div_year:]))

### plot figures
ax = plt.subplot(1,2,1)
mean_ratio = np.mean(my_run['VT'][div_year:])/np.mean(my_run['VT'][:div_year])
std_ratio = np.std(my_run['VT'][div_year:])/np.std(my_run['VT'][:div_year])
vmin = np.nanmin(np.hstack((mean_intel,mean_test)))
vmax = np.nanmax(np.hstack((mean_intel,mean_test)))
nbins = 16
bins = np.linspace(vmin,vmax,nbins)
ax.hist(mean_intel,bins,color = 'b',label = 'Intel',alpha = 0.4,density = True)
ax.hist(mean_test,bins,color = 'orange',label = my_test,alpha = 0.4,density = True)
ax.set_title('Mean')
#p = plt.hist(mean_intel,bins=bins)[0]
#q = plt.hist(mean_test,bins=bins)[0]
#p = p/p.sum(axis=0, keepdims=True) 
#q = q/q.sum(axis=0, keepdims=True) 

#plt.hist(p,bins)
#plt.hist(q,bins)
ylim = plt.gca().get_ylim()
plt.vlines(mean_ratio,0,ylim[1],linestyle = '--')


ax = plt.subplot(1,2,2)
vmin = np.nanmin(np.hstack((std_intel,std_test)))
vmax = np.nanmax(np.hstack((std_intel,std_test)))
bins = np.linspace(vmin,vmax,nbins)
ax.set_title('Stddev')

ax.hist(std_intel,bins,color = 'b',label = 'Intel',alpha = 0.4,density = True)
ax.hist(std_test,bins,color = 'orange',label = my_test,alpha = 0.4,density = True)
ylim = plt.gca().get_ylim()
plt.vlines(std_ratio,0,ylim[1],linestyle = '--')

plt.legend()


Randomly draw 10 years from the distribution, how likely  is the first 10 years of Intel/GNU/Test runs lie outside the distribution? 

In [ ]:
len(var_list)

In [ ]:
### Shuffle Intel
import matplotlib.ticker as ticker

my_test = 'rh-min-low'
intel_df = df[df.Label == 'Intel']
new_var_list = ['TS','TSMN','TSMX','VT','VU','U10','LWCF','BURDEN2','CCN3']
#new_var_list[6] = 'SWCF'

my_intel_simu = '001'


first_nyears = 10

fig = plt.figure(1,figsize = (12,12))

#std_intel = []
nyear = 100
use_year = 90

my_run = df[(df.Simulation == my_test) & (df.nyear >= nyear - use_year)]
my_intel = intel_df[(intel_df.Simulation == my_intel_simu) & (intel_df.nyear >= nyear - use_year)]
#my_var = 'BURDEN2'
nshuffle = 10
div_year = int(use_year/2)

for ivar,my_var in enumerate(new_var_list):
  ten_yr_intel = []
  ax = plt.subplot(3,3,ivar+1)
  for my_simu in np.unique(intel_df.Simulation):
    intel_ts = np.array(intel_df[(intel_df.Simulation == my_simu) & (intel_df.nyear >= nyear - use_year)][my_var])
    for ishuffle in range(nshuffle):
      my_array = np.random.choice(range(use_year),first_nyears,replace = False)
      new_ts = intel_ts.copy()    
      ten_yr_intel.append(np.nanmean(new_ts[my_array]))

  ### Shuffle rh-min-low
  ten_yr_test = []
  rh_ts = np.array(my_run[my_var])
  print(rh_ts)
  nshuffle_test = 100
  for ishuffle in range(nshuffle_test):
    my_array = np.random.choice(range(use_year),first_nyears,replace = False)
    new_ts = rh_ts.copy()
    ten_yr_test.append(np.nanmean(new_ts[my_array]))

  ### plot figures
  test_raw = np.nanmean(my_run[my_var][:first_nyears])
  intel_raw = np.nanmean(my_intel[my_var][:first_nyears])
  #std_ratio = np.std(my_run[my_var][div_year:])/np.std(my_run[my_var][:div_year])
  vmin = np.nanmin(np.hstack((ten_yr_intel,ten_yr_test)))
  vmax = np.nanmax(np.hstack((ten_yr_intel,ten_yr_test)))
  nbins = 16
  bins = np.linspace(vmin,vmax,nbins)
  ax.hist(ten_yr_intel,bins,color = 'k',label = 'Intel',alpha = 0.4,density = True)
  ax.hist(ten_yr_test,bins,color = 'orange',label = my_test,alpha = 0.4,density = True)
  #ax.set_title('Mean')
  #p = plt.hist(mean_intel,bins=bins)[0]
  #q = plt.hist(mean_test,bins=bins)[0]
  #p = p/p.sum(axis=0, keepdims=True) 
  #q = q/q.sum(axis=0, keepdims=True) 

  #plt.hist(p,bins)
  #plt.hist(q,bins)
  ylim = plt.gca().get_ylim()
  plt.vlines(intel_raw,0,ylim[1],linestyle = '--')
  plt.vlines(test_raw,0,ylim[1],linestyle = '-.')
  plt.xlabel(my_var)
  if ivar < 3:
    ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('%0.1f'))

  #plt.xticks(ticks=4)
  if ivar % 3 == 0:
    plt.ylabel('Histogram')
  if ivar == 8:
    plt.legend()


In [ ]:
intel_raw = np.nanmean(my_intel[my_var][:first_nyears])
intel_raw


In [ ]:
df.Simulation

In [ ]:
### Shuffle Intel
my_test = 'rh-min-low'
my_run = df[(df.Simulation == my_test)]


fig = plt.figure(1,figsize = (12,6))
mean_intel = []
std_intel = []
nyear = 100
use_year = 100
my_var = 'BURDEN2'
intel_df = df[df.Label == 'Intel']
nshuffle = 10
div_year = int(use_year/2)

for my_simu in np.unique(intel_df.Simulation):
  intel_ts = np.array(intel_df[(intel_df.Simulation == my_simu) & (intel_df.nyear >= nyear - use_year)][my_var])
  for ishuffle in range(nshuffle):
    np.random.shuffle(intel_ts)
    new_ts = intel_ts.copy()
    
    mean_intel.append(np.nanmean(new_ts[:div_year])/np.nanmean(new_ts[div_year:]))
    std_intel.append(np.nanstd(new_ts[:div_year])/np.nanstd(new_ts[div_year:]))

### Shuffle rh-min-low
mean_test = []
std_test = []
rh_ts = np.array(my_run[my_var])
print(rh_ts)
for ishuffle in range(1000):
  np.random.shuffle(rh_ts)
  new_ts = rh_ts.copy()
  mean_test.append(np.nanmean(new_ts[:div_year])/np.nanmean(new_ts[div_year:]))
  std_test.append(np.nanstd(new_ts[:div_year])/np.nanstd(new_ts[div_year:]))

### plot figures
ax = plt.subplot(1,2,1)
mean_ratio = np.mean(my_run[my_var][div_year:])/np.mean(my_run[my_var][:div_year])
std_ratio = np.std(my_run[my_var][div_year:])/np.std(my_run[my_var][:div_year])
vmin = np.nanmin(np.hstack((mean_intel,mean_test)))
vmax = np.nanmax(np.hstack((mean_intel,mean_test)))
nbins = 16
bins = np.linspace(vmin,vmax,nbins)
ax.hist(mean_intel,bins,color = 'b',label = 'Intel',alpha = 0.4,density = True)
ax.hist(mean_test,bins,color = 'orange',label = my_test,alpha = 0.4,density = True)
ax.set_title('Mean')
#p = plt.hist(mean_intel,bins=bins)[0]
#q = plt.hist(mean_test,bins=bins)[0]
#p = p/p.sum(axis=0, keepdims=True) 
#q = q/q.sum(axis=0, keepdims=True) 

#plt.hist(p,bins)
#plt.hist(q,bins)
ylim = plt.gca().get_ylim()
plt.vlines(mean_ratio,0,ylim[1],linestyle = '--')


ax = plt.subplot(1,2,2)
vmin = np.nanmin(np.hstack((std_intel,std_test)))
vmax = np.nanmax(np.hstack((std_intel,std_test)))
ax.set_title('Stddev')
#bins = np.linspace(vmin,vmax,nbins)

bins = np.logspace(np.log(vmin),np.log(vmax),nbins)
plt.xscale('log')


ax.hist(std_intel,bins,color = 'b',label = 'Intel',alpha = 0.4,density = True)
ax.hist(std_test,bins,color = 'orange',label = my_test,alpha = 0.4,density = True)
ylim = plt.gca().get_ylim()
plt.vlines(std_ratio,0,ylim[1],linestyle = '--')

plt.legend()


In [ ]:
### How many variables fails the 5% KS-test in each year (or plot p-value across time)
nyear = 100
var_list = ['TS','TSMN','TSMX','VT','VU','U10','LWCF','BURDEN2','CCN3']

confidence_level = 0.05
zscore_ts = np.zeros((len(test_list),len(var_list),nyear))
std_intel_ts = np.zeros((len(test_list),len(var_list),nyear))
std_gnu_ts = np.zeros((len(test_list),len(var_list),nyear))
gnu_df = df[df.Label == 'GNU']

for itest, my_test in enumerate(test_list):
  for yr in range(nyear):
    num_fail = 0
    for ivar,my_var in enumerate(var_list):
      x1 = intel_df[(intel_df.nyear == yr)][my_var]
      x2 = gnu_df[(gnu_df.nyear == yr)][my_var]
      y = df[(df.Simulation.str.contains(my_test)) & (df.nyear == yr)][my_var]
      z_score = (y - x1.mean())/x1.std()
      std_intel_ts[itest,ivar,yr] = x1.std()
      std_gnu_ts[itest,ivar,yr] = x2.std()
      zscore_ts[itest,ivar,yr] = z_score



In [ ]:

zscore_intel = np.zeros((2,len(var_list),nyear)) # Dim 0: upper limit for 0, and lower limit for 1

for yr in range(nyear):
  for ivar,my_var in enumerate(var_list):
    x1 = intel_df[(intel_df.nyear == yr)][my_var]
    #x2 = gnu_df[(gnu_df.nyear == yr)][my_var]
    for isimu, my_quant in enumerate([0.975,0.025]):
      y = x1.quantile(my_quant)
      z_score = (y - x1.mean())/x1.std()
      zscore_intel[isimu,ivar,yr] = z_score


In [ ]:
itest = 0
zscore_table = pd.DataFrame(columns =  var_list)
for itest,my_test in enumerate(test_list):
  for irow,all_year in enumerate([1,10,50,100]):
    empty_array = np.zeros(len(var_list))
    for ivar, my_var in enumerate(var_list):
      #empty_array[ivar] = round(np.mean(abs(zscore_ts[itest,ivar,:nyear])),3)
      empty_array[ivar] = round(np.mean((zscore_ts[itest,ivar,:all_year])),3)
    one_line = pd.DataFrame(columns = var_list, data = empty_array.reshape(1,9))
    one_line['nyear'] = int(all_year)
    one_line['Simulation'] = test_list[itest]
    
    zscore_table = zscore_table.append(one_line,ignore_index = True)
zscore_table.set_index(['Simulation','nyear']).reindex(columns = var_list) 

In [ ]:
itest= 0
idx = 0
plt.figure(1,figsize = (12,8))
#for itest,my_test in enumerate(test_list):
sns.lineplot(range(nyear),abs(zscore_ts[itest,idx,:]),label = test_list[itest])
plt.hlines([0],0,100,linestyle = '--')
plt.xlabel("Years",fontsize = fs)
plt.ylabel("Z-score",fontsize = fs)
plt.legend(frameon=False,fontsize = fs)
plt.title(var_list[idx])

In [ ]:
fig = plt.figure(1,figsize = (8,8))
itest0 = 1
idx0 = 0
idx1 = 6
fs = 14
sns.kdeplot(zscore_ts[itest0,idx0,:],zscore_ts[itest0,idx1,:],shade = True)
plt.xlabel('Z-score '+var_list[idx0],fontsize = fs)
plt.ylabel('Z-score '+var_list[idx1],fontsize = fs)

plt.scatter(zscore_ts[itest0,idx0,0],zscore_ts[itest0,idx1,0],marker= 'o',c = 'r')
xlim = plt.gca().get_xlim()
ylim = plt.gca().get_ylim()

plt.hlines([0],xlim[0],xlim[1],linestyle = '--')
plt.vlines([0],ylim[0],ylim[1],linestyle = '--')

In [ ]:
fig = plt.figure(1,figsize = (8,8))
itest0 = 1
idx0 = 0
idx1 = 6
fs = 14
sns.kdeplot(zscore_ts[itest0,idx0,:],zscore_ts[itest0,idx1,:],shade = True)
plt.xlabel('Z-score '+var_list[idx0],fontsize = fs)
plt.ylabel('Z-score '+var_list[idx1],fontsize = fs)

all_year = 1

for itest0,my_test,my_marker, my_color in zip(range(8),test_list,['o','o','o','o','*','*','*','*'],['k','r','b','g','k','r','b','g']):
  plt.scatter(np.mean(zscore_ts[itest0,idx0,:all_year]),np.mean(zscore_ts[itest0,idx1,:all_year]),marker= my_marker,c = my_color)
xlim = plt.gca().get_xlim()
ylim = plt.gca().get_ylim()

plt.hlines([0],xlim[0],xlim[1],linestyle = '--')
plt.vlines([0],ylim[0],ylim[1],linestyle = '--')


In [ ]:

### 9 subplots: How do standard deviation evolve in Intel ensemble?
### Z scores changing with year
normalize_y = False
use_year = 100
nrow = 3
ncol = 3
fig = plt.figure(figsize=(12,12))
plt.subplots_adjust(wspace = 0.4,hspace= 0.3)

#fig = plt.figure(figsize=(8,8))

plt.tight_layout()
# Set to True to use only 25 simulations 
nyear =100

for irow in range(nrow):
  for icol in range(ncol):
    idx = ncol*irow+icol
    ax = plt.subplot(nrow,ncol,idx+1)
    my_var = var_list[idx]
    #for itest,my_test in enumerate(test_list):
    itest = 0
    plt.plot(range(nyear),std_intel_ts[itest,idx,:],label = 'Intel')
    plt.plot(range(nyear),std_gnu_ts[itest,idx,:],label = 'GNU')
    if irow == 2:
      plt.xlabel('Year',fontsize = fs)
    if icol == 0:
      plt.ylabel('Internal Variability',fontsize=fs)
    plt.title(my_var)
    if irow == 0 and icol == 2:
      ax.legend(frameon=False)

### Plot z-score for test runs
fig = plt.figure(2,figsize=(12,12))
plt.subplots_adjust(wspace = 0.4,hspace= 0.3)

plt.tight_layout()
# Set to True to use only 25 simulations 


for irow in range(nrow):
  for icol in range(ncol):
    idx = ncol*irow+icol
    ax = plt.subplot(nrow,ncol,idx+1)
    my_var = var_list[idx]
    #for itest,my_test in enumerate(test_list):
    itest0 = 0
    itest1 = 1
    plt.plot(range(nyear),zscore_ts[itest0,idx,:],label = test_list[itest0])
    plt.plot(range(nyear),zscore_ts[itest1,idx,:],label = test_list[itest1])
    plt.xlabel('Year')
    plt.ylabel('Z-score')
    plt.title(my_var)
    upper = 0.975
    lower = 0.025
    up_limit = zscore_intel[0,idx]
    lw_limit = zscore_intel[1,idx]
    #plt.plot(range(nyear),up_limit,linestyle = '--',color='k')
    #plt.plot(range(nyear),lw_limit,linestyle = '--',color='k')
    if irow == 0 and icol == 2:
      ax.legend(frameon=False)


In [ ]:
### 9 subplots
normalize_y = False
abs_value = True # Set to True to use absolute z-score
nrow = 3
ncol = 3
fig = plt.figure(figsize=(12,12))
plt.subplots_adjust(wspace = 0.4,hspace= 0.3)

#fig = plt.figure(figsize=(8,8))

plt.tight_layout()
# Set to True to use only 25 simulations 


for irow in range(nrow):
  for icol in range(ncol):
    idx = ncol*irow+icol
    ax = plt.subplot(nrow,ncol,idx+1)
    my_var = var_list[idx]
    #for itest,my_test in enumerate(test_list):
    #itest = 0
    for itest in [0,7]:
      if abs_value:
        plt.plot(range(nyear),abs(zscore_ts[itest,idx,:]),label = test_list[itest])
        #plt.ylim([-0.5,4.2])
        plt.locator_params(axis='y', nbins=5)

      else:
        plt.plot(range(nyear),(zscore_ts[itest,idx,:]),label = test_list[itest])
        plt.ylim([-3.5,3.5])

      plt.hlines([0],0,100,linestyle = '--')
      plt.xlabel('Year')
      plt.ylabel('Z-score')
      plt.title(my_var)
      if irow == 0 and icol == 2:
        ax.legend(frameon=False)


#### Same standard deviation and mean

 To derive for Z-score of each test run

In [ ]:
### 9 subplots, Z-scores that derived based on 100 year mean and standard deviation of Intel ensemble


'''
confidence_level = 0.05
zscore_ts = np.zeros((len(test_list),len(var_list),nyear))
for ivar,my_var in enumerate(var_list):
  x1 = intel_df[my_var]
  for itest, my_test in enumerate(test_list):
    for yr in range(nyear):
      y = df[(df.Simulation.str.contains(my_test)) & (df.nyear == yr)][my_var]
      z_score = (y - x1.mean())/x1.std()     
      zscore_ts[itest,ivar,yr] = z_score
'''

In [ ]:
'''
normalize_y = False
abs_value = False # Set to True to use absolute z-score
nrow = 3
ncol = 3

fig = plt.figure(figsize=(12,12))
plt.subplots_adjust(wspace = 0.4,hspace= 0.3)

#fig = plt.figure(figsize=(8,8))

plt.tight_layout()
for irow in range(nrow):
  for icol in range(ncol):
    idx = ncol*irow+icol
    ax = plt.subplot(nrow,ncol,idx+1)
    my_var = var_list[idx]
    #for itest,my_test in enumerate(test_list):
    #itest = 0
    for itest in [0,7]:
      if abs_value:
        plt.plot(range(nyear),abs(zscore_ts[itest,idx,:]),label = test_list[itest])
        #plt.ylim([-0.5,4.2])
        plt.locator_params(axis='y', nbins=5)

      else:
        plt.plot(range(nyear),(zscore_ts[itest,idx,:]),label = test_list[itest])
        #plt.ylim([-3.5,3.5])

      plt.hlines([0],0,100,linestyle = '--')
      plt.xlabel('Year')
      plt.ylabel('Z-score')
      plt.title(my_var)
      if irow == 0 and icol == 2:
        ax.legend(frameon=False)
'''

In [ ]:
#np.argwhere(zscore_ts[:,:,0] == np.max(zscore_ts[:,:,0]))
variance_ratio = np.zeros((len(var_list),len(np.unique(intel_df.Simulation))))
for isimu, my_simu in enumerate(np.unique(intel_df.Simulation)):
  newdf = intel_df[(intel_df.Simulation == my_simu)]
  for ivar, my_var in enumerate(var_list):
    variance_ratio[ivar,isimu] = (newdf[newdf.nyear >= 50][my_var].std())/(newdf[newdf.nyear < 50][my_var].std())

test_variance_ratio = np.zeros((len(var_list),8))
for isimu, my_simu in enumerate(test_list):
  newdf = df[df.Simulation == test_list[isimu]]
  for ivar, my_var in enumerate(var_list):
    test_variance_ratio[ivar,isimu] = (newdf[newdf.nyear >= 50][my_var].std())/(newdf[newdf.nyear < 50][my_var].std())



In [ ]:
test_variance_ratio

In [ ]:
idx = 3
itest = 1
plt.hist(variance_ratio[idx,:])



print(test_variance_ratio[idx,itest])

for idx,my_var in enumerate(var_list):
  for itest,my_test in enumerate(test_list):
    if test_variance_ratio[idx,itest] < np.quantile(variance_ratio[idx,:],0.025) or test_variance_ratio[idx,itest] > np.quantile(variance_ratio[idx,:],0.975):
      print('This is '+str(my_var)+','+str(my_test))


In [ ]:
fig = plt.figure(figsize=(10,6))
ax2 = plt.subplot()
ax2.set_title('Ratio of standard deviation (Second half/First half)')
data = [variance_ratio[idx,:] for idx in range(9)]
ax2.boxplot(data, notch=True,showfliers=True,labels = var_list)
for idx, my_var in enumerate(var_list):
  ax2.scatter(idx+1,test_variance_ratio[idx,7],marker = 'o',c= 'r')

#sns.boxplot(variance_ratio[:,:])

In [ ]:
idx = 0
itest = 1
yr_len = 50
bins = np.linspace(-3,3,20)
plt.hist(zscore_ts[itest,idx,:yr_len],bins = bins, density = True,alpha = 0.3,label = 'First half')#histtype='step')
plt.hist(zscore_ts[itest,idx,-yr_len:],bins = bins, density = True,alpha= 0.3,label = 'Second half')#,histtype='step')

plt.vlines(zscore_ts[itest,idx,1], 0, 0.4, linestyle = '--')
plt.legend(frameon=False)

#### Correlation coefficients

Same variable, across simulations. 

In [ ]:
var_df
new_var_df = var_df.sort_values(by='type').reset_index()
new_var_df
